In [ ]:
import cv2 as cv
from tqdm import tqdm
import pandas as pd
from collections.abc import Iterable
from pims import *
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from pylab import *
from skimage.io import imread
import imageio
from glob import glob
from cellocity.channel import Channel
import tifffile
from tifffile import imshow
from time import time
import warnings
import plotly.express as px
import dabest
import seaborn as sns
from skimage.filters import difference_of_gaussians, window
from scipy.fft import fftn, fftshift
import plotly
from scipy import stats
from skimage import exposure#, filters, util
# import skimage
from openpiv import preprocess as piv_pre
import openpiv.tools as piv_tls
from datetime import datetime
import math
import pickle
import plotly.graph_objects as go
import builtins


In [ ]:
from openpiv import tools, pyprocess, validation, filters, scaling, piv, preprocess
from openpiv.pyprocess import *
import openpiv.pyprocess as process
from openpiv import pyprocess
from openpiv.pyprocess import extended_search_area_piv, get_field_shape, get_coordinates
from openpiv import smoothn
from openpiv.preprocess import mask_coordinates
from openpiv.piv import simple_piv
from openpiv.tools import imread, Multiprocesser, display_vector_field, \
    transform_coordinates
from openpiv import validation, filters, tools, preprocess, scaling, tools
from skimage.measure import points_in_poly
from skimage.util import invert
from openpiv import windef
from openpiv.windef import first_pass, multipass_img_deform


In [ ]:
def piv_run(frame_a, frame_b, mask_a, mask_b, settings, file_name, counter):
    # "first pass"
    x, y, u, v, s2n = first_pass(frame_a, frame_b, settings)

    if settings.show_all_plots:
        plt.figure()
        plt.quiver(x,y,u,-v,color='b')
        # plt.gca().invert_yaxis()
        # plt.gca().set_aspect(1.)
        # plt.title('after first pass, invert')
        # plt.show()

    # " Image masking "
    if settings.image_mask:
        image_mask = np.logical_and(mask_a, mask_b)
        mask_coords = preprocess.mask_coordinates(image_mask)
        # mark those points on the grid of PIV inside the mask
        grid_mask = preprocess.prepare_mask_on_grid(x,y,mask_coords)

        # mask the velocity
        u = np.ma.masked_array(u, mask=grid_mask)
        v = np.ma.masked_array(v, mask=grid_mask)
    else:
        mask_coords = []
        u = np.ma.masked_array(u, mask=np.ma.nomask)
        v = np.ma.masked_array(v, mask=np.ma.nomask)

    if settings.validation_first_pass:
        u, v, mask = validation.typical_validation(u, v, s2n, settings)

    if settings.show_all_plots:
        plt.figure()
        plt.quiver(x,y,u,-v,color='r')
        plt.gca().invert_yaxis()
        plt.gca().set_aspect(1.)
        plt.title('after first pass validation new, inverted')
        plt.show()


    # "filter to replace the values that where marked by the validation"
    if settings.num_iterations == 1 and settings.replace_vectors:
        # for multi-pass we cannot have holes in the data
        # after the first pass
        u, v = filters.replace_outliers(
            u,
            v,
            method=settings.filter_method,
            max_iter=settings.max_filter_iteration,
            kernel_size=settings.filter_kernel_size,
        )
    elif settings.num_iterations > 1: # don't even check if it's true or false
        u, v = filters.replace_outliers(
            u,
            v,
            method=settings.filter_method,
            max_iter=settings.max_filter_iteration,
            kernel_size=settings.filter_kernel_size,
        )

        # "adding masks to add the effect of all the validations"
    if settings.smoothn:
        u, dummy_u1, dummy_u2, dummy_u3 = smoothn.smoothn(u, s=settings.smoothn_p)
        v, dummy_v1, dummy_v2, dummy_v3 = smoothn.smoothn(v, s=settings.smoothn_p)

    if settings.image_mask:
        grid_mask = preprocess.prepare_mask_on_grid(x, y, mask_coords)
        u = np.ma.masked_array(u, mask=grid_mask)
        v = np.ma.masked_array(v, mask=grid_mask)
    else:
        u = np.ma.masked_array(u, np.ma.nomask)
        v = np.ma.masked_array(v, np.ma.nomask)


    if settings.show_all_plots:
        plt.figure()
        plt.quiver(x,y,u,-v)
        plt.gca().invert_yaxis()
        plt.gca().set_aspect(1.)
        plt.title('before multi pass, inverted')
        plt.show()

    if not isinstance(u, np.ma.MaskedArray):
        raise ValueError("Expected masked array")

    """ Multi pass """
    for i in range(1, settings.num_iterations):
        if not isinstance(u, np.ma.MaskedArray):
            raise ValueError("Expected masked array")
        x, y, u, v, s2n, mask = multipass_img_deform(
            frame_a,
            frame_b,
            i,
            x,
            y,
            u,
            v,
            settings,
            mask_coords=mask_coords
        )   
        
        # If the smoothing is active, we do it at each pass
        # but not the last one
        if settings.smoothn is True and i < settings.num_iterations-1: 
            u, dummy_u1, dummy_u2, dummy_u3 = smoothn.smoothn(u, s=settings.smoothn_p)
            v, dummy_v1, dummy_v2, dummy_v3 = smoothn.smoothn(v, s=settings.smoothn_p)
        if not isinstance(u, np.ma.MaskedArray):
            raise ValueError ('not a masked array anymore')

        if hasattr(settings, 'image_mask') and settings.image_mask:
            grid_mask = preprocess.prepare_mask_on_grid(x, y, mask_coords)
            u = np.ma.masked_array(u, mask=grid_mask)
            v = np.ma.masked_array(v, mask=grid_mask)
        else:
            u = np.ma.masked_array(u, np.ma.nomask)
            v = np.ma.masked_array(v, np.ma.nomask)

        if settings.show_all_plots:
            plt.figure()
            plt.quiver(x, y, u, -v, color='r')
            plt.gca().set_aspect(1.)
            plt.gca().invert_yaxis()
            plt.title('end of the multipass, invert')
            plt.show()

    if settings.show_all_plots and settings.num_iterations > 1:
        plt.figure()
        plt.quiver(x,y,u,-v)
        plt.gca().invert_yaxis()
        plt.gca().set_aspect(1.)
        plt.title('after multi pass, before saving, inverted')
        plt.show()

    # we now use only 0s instead of the image
    # masked regions. 
    # we could do Nan, not sure what is best
    u = u.filled(0.)
    v = v.filled(0.)

    # "scales the results pixel-> meter"
    x, y, u, v = scaling.uniform(x, y, u, v, scaling_factor=settings.scaling_factor)

    if settings.image_mask:
        grid_mask = preprocess.prepare_mask_on_grid(x, y, mask_coords)
        u = np.ma.masked_array(u, mask=grid_mask)
        v = np.ma.masked_array(v, mask=grid_mask)
    else:
        u = np.ma.masked_array(u, np.ma.nomask)
        v = np.ma.masked_array(v, np.ma.nomask)

    # before saving we conver to the "physically relevant"
    # right-hand coordinate system with 0,0 at the bottom left
    # x to the right, y upwards
    # and so u,v

    x, y, u, v = transform_coordinates(x, y, u, v)
    # import pdb; pdb.set_trace()
    # "save to a file"
    tools.save(x, y, u, v, mask,
        os.path.join(settings.save_path, "field_%s_%03d.txt" %(file_name,counter)),
        delimiter="\t",
    )
    
        # "some other stuff that one might want to use"
    if settings.show_plot or settings.save_plot:
        Name = os.path.join(settings.save_path, "PIV_%s_%03d.png" %(file_name,counter))
        fig, _ = display_vector_field(
            os.path.join(settings.save_path, "field_%s_%03d.txt" %(file_name,counter)),
            scale=settings.scale_plot,
        )
        if settings.save_plot is True:
            fig.savefig(Name)
        if settings.show_plot is True:
            plt.show()
    return x, y, u, v


In [ ]:
def piv(settings, file_name, mask_a, mask_b, counter):
    """A function to process each image pair."""

    frame_a = settings.frame_pattern_a
    frame_b = settings.frame_pattern_b

    " crop to ROI"
    if settings.ROI == "full":
        frame_a = frame_a
        frame_b = frame_b
    else:
        frame_a = frame_a[
            settings.ROI[0]:settings.ROI[1],
            settings.ROI[2]:settings.ROI[3]
        ]
        frame_b = frame_b[
            settings.ROI[0]:settings.ROI[1],
            settings.ROI[2]:settings.ROI[3]
        ]

    if settings.invert is True:
        frame_a = invert(frame_a)
        frame_b = invert(frame_b)

    if settings.show_all_plots:
        fig, ax = plt.subplots(1,1)
        ax.imshow(frame_a, cmap=plt.get_cmap('Reds'))
        ax.imshow(frame_b, cmap=plt.get_cmap('Blues'),alpha=.5)
        plt.show()

    if settings.dynamic_masking_method in ("edge", "intensity"):
        frame_a, mask_a = preprocess.dynamic_masking(
            frame_a,
            method=settings.dynamic_masking_method,
            filter_size=settings.dynamic_masking_filter_size,
            threshold=settings.dynamic_masking_threshold,
        )
        frame_b, mask_b = preprocess.dynamic_masking(
            frame_b,
            method=settings.dynamic_masking_method,
            filter_size=settings.dynamic_masking_filter_size,
            threshold=settings.dynamic_masking_threshold,
        )

    x, y, u, v = piv_run(frame_a, frame_b, mask_a, mask_b, settings, file_name, counter)    
    return x, y, u, v


In [ ]:
settings = windef.Settings()

'Region of interest'
settings.ROI = 'full'

'Image preprocessing'
# 'None' for no masking, 'edges' for edges masking, 'intensity' for intensity masking
settings.dynamic_masking_method = 'edges'
settings.dynamic_masking_threshold = 0.5
settings.dynamic_masking_filter_size = 21

# settings.deformation_method = 'symmetric' 
settings.deformation_method = 'second image'

'Processing Parameters'
settings.correlation_method='linear'  # 'circular' or 'linear'
settings.normalized_correlation = True

settings.num_iterations = 4  # select the number of PIV passes
settings.windowsizes=(64, 64, 32, 32, 16, 16)
settings.overlap=(32, 32, 16, 16, 8, 8)

# Has to be a value with base two. In general window size/2 is a good choice.
# method used for subpixel interpolation: 'gaussian','centroid','parabolic'
settings.subpixel_method = 'gaussian'

# order of the image interpolation for the window deformation
settings.interpolation_order = 3
settings.scaling_factor = 1  # scaling factor pixel/meter
settings.dt = 1  # time between to frames (in seconds)

'Signal to noise ratio options (only for the last pass)'
# It is possible to decide if the S/N should be computed (for the last pass) or not
# settings.extract_sig2noise = True  # 'True' or 'False' (only for the last pass)
settings.sig2noise_threshold = 1.3
# method used to calculate the signal to noise ratio 'peak2peak' or 'peak2mean'
settings.sig2noise_method = 'peak2mean'
# select the width of the masked to masked out pixels next to the main peak
settings.sig2noise_mask = 2
settings.sig2noise_validate = False

'vector validation options'
# choose if you want to do validation of the first pass: True or False
settings.validation_first_pass = True

'Validation Parameters'
# The validation is done at each iteration based on three filters.
# The first filter is based on the min/max ranges. Observe that these values are defined in
# terms of minimum and maximum displacement in pixel/frames.
settings.MinMax_U_disp = (-5, 5)
settings.MinMax_V_disp = (-5, 5)
# The second filter is based on the global STD threshold
settings.std_threshold = 2  # threshold of the std validation
# The third filter is the median test (not normalized at the moment)
settings.median_threshold = 2  # threshold of the median validation
# On the last iteration, an additional validation can be done based on the S/N.
settings.median_size = 2 # defines the size of the local median

'Outlier replacement or Smoothing options'
# Replacment options for vectors which are masked as invalid by the validation
settings.replace_vectors = True # Enable the replacment. Chosse: True or False
settings.smoothn = True #Enables smoothing of the displacemenet field
settings.smoothn_p = 0.5 # This is a smoothing parameter
# select a method to replace the outliers: 'localmean', 'disk', 'distance'
settings.filter_method = 'localmean'
# maximum iterations performed to replace the outliers
settings.max_filter_iteration = 1
settings.filter_kernel_size = 1  # kernel size for the localmean method

'Output options'
# Select if you want to save the plotted vectorfield: True or False
settings.save_plot = False
# Choose wether you want to see the vectorfield or not :True or False
settings.show_plot = False
settings.scale_plot = 20  # select a value to scale the quiver plot of the vectorfield

settings.image_mask = True
settings.show_all_plots = False
settings.invert = False


In [ ]:
# This function calculates the variability in velocity direction by quantifying angular deviation
# angular deviation is calculated as 2(√1−z) where z=1/N[(∑cosθ)^2+(∑sinθ)^2]1/2
# function takes a set of angles as input
# function returns array with angular deviation of array

def calc_ang_dev(angles):
    non_nan = np.array(angles)[~np.isnan(angles)]
    term1 = sum(np.cos(non_nan))/np.size(non_nan)
    term2 = sum(np.sin(non_nan))/np.size(non_nan)
    z = np.sqrt(np.square(term1)+np.square(term2))
    ang_dev = np.sqrt(2*(1-z))
    return ang_dev

In [ ]:
def ang_btwn(p1, p2):
    dx=p2[0]-p1[0]
    dy=p2[1]-p1[1]
    return np.arctan2(dy,dx)

In [ ]:
def straighten_scratch(binary_image):
    cnt = cv.findContours(~binary_image[0], cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)[0]
    for k in range(len(cnt)):
        area = cv.contourArea(cnt[k])
        if area>2000:
            # grab the (x, y) coordinates of all border values
            otln = np.squeeze(cnt[k])
            min_x = int(min(otln.T[0]))
            min_y = int(min(otln.T[1]))
            max_x = int(max(otln.T[0]))
            max_y = int(max(otln.T[1]))
            dl=[]
            for j in range(len(otln)):
                if otln.tolist()[j][0] == min_x or otln.tolist()[j][0] == max_x or otln.tolist()[j][1] == min_y or otln.tolist()[j][1] == max_y:
                    dl.append(j)

            # We will than be deleting these values from our contour array so that we are only
            # looking at the monolayer edge boundarys
            rslt = np.delete(otln, dl, 0)

            # compute a rotated bounding box that contains all coordinates
            box_ang  = cv.minAreaRect(rslt)[-1]

            # the `cv2.minAreaRect` function returns values in the
            # range [-90, 0); as the rectangle rotates clockwise the
            # returned angle trends to 0
            if box_ang > 45:
                box_ang = (90 - box_ang)
            # otherwise, just take the inverse of the angle to make it positive
            else:
                box_ang = -box_ang
#             box_ang = np.deg2rad(box_ang)
    return box_ang

In [ ]:
def rotate_matrix (x, y, angle, x_shift=0, y_shift=0, units="DEGREES"):
    """
    Rotates a point in the xy-plane counterclockwise through an angle about the origin
    https://en.wikipedia.org/wiki/Rotation_matrix
    :param x: x coordinate
    :param y: y coordinate
    :param x_shift: x-axis shift from origin (0, 0)
    :param y_shift: y-axis shift from origin (0, 0)
    :param angle: The rotation angle in degrees
    :param units: DEGREES (default) or RADIANS
    :return: Tuple of rotated x and y
    """

    # Shift to origin (0,0)
    x = x - x_shift
    y = y - y_shift

    # Convert degrees to radians
    if units == "DEGREES":
        angle = math.radians(angle)

    # Rotation matrix multiplication to get rotated x & y
    xr = (x * math.cos(angle)) - (y * math.sin(angle)) + x_shift
    yr = (x * math.sin(angle)) + (y * math.cos(angle)) + y_shift

    return xr, yr

In [ ]:
#define cohens d using the function defined here: https://machinelearningmastery.com/effect-size-measures-in-python/
def cohend(d1, d2):
    # calculate the size of samples
    n1, n2 = len(d1), len(d2)
    # calculate the variance of the samples
    s1, s2 = np.nanvar(d1, ddof=1), np.nanvar(d2, ddof=1)
    # calculate the pooled standard deviation
    s = np.sqrt(((n1 - 1) * s1 + (n2 - 1) * s2) / (n1 + n2 - 2))
    # calculate the means of the samples
    u1, u2 = np.nanmean(d1), np.nanmean(d2)
    # calculate the effect size
    return (u2 - u1) / s

In [ ]:
def get_dists(X, Y, r_bin_size):
    # Calculate all possible distances within a FOV
    XY = np.dstack((X[0],Y[0]))
    XY_flat = XY.reshape(-1, XY.shape[-1])
    a, b, c = np.shape(XY)
    dists = []

    for i in range(a):
        cols = []
        for j in range(b):
            point = XY[i, j]
            shp = []
            for k in range(len(XY_flat)):
                pnt_dist = np.linalg.norm(XY_flat[k]-point)
                shp.append(pnt_dist)
            taco = np.reshape(shp, (a,b))
            cols.append(taco)
        dists.append(cols)

    max_dist = min(np.nanmax(dists), np.inf)
    r_bins = np.arange(0, max_dist, r_bin_size)
    
    return dists, r_bins

In [ ]:
def spatial_coord_variability(v_rho, X_scaled, Y_scaled, r_bin_size):
    'Calculate all possible distances within a FOV'
    dists, r_bins = get_dists(X_scaled, Y_scaled, r_bin_size)
    
    Cr_jj = []
    N_jj = []
    Cr_spatial_jj={}
    Cr_m_jj = []
    Cr_std_jj = []

    for jj in range(len(r_bins)):
        if jj == 1:
            r, s, t, u = np.where(dists == r_bins[jj])
        else:
            r, s, t, u = np.where((dists > r_bins[jj-1]) & (dists < r_bins[jj]))

        Cr_kk = []
        N_kk = []
        Cr_spatial_kk = []
        Cr_m_kk = []
        Cr_std_kk = []
        for kk in range(len(piv_val)):
            im = piv_val[kk]
            im1 = []
            im2 = []
            for q in range(len(r)):
                im_1 = im[r[q], s[q]]
                im_2 = im[t[q], u[q]]
                im1.append(im_1)
                im2.append(im_2)

            # makes sure the terms have same number of unmasked values
            mask = np.divide(np.multiply(im1, im2), np.multiply(im1, im2))
            im1 = np.multiply(mask, im1)
            im2 = np.multiply(mask, im2)

            corr_temp = np.multiply(im1, im2)        
            term1 = np.square(im1)
            term1 = np.nansum(term1)
            term2 = np.square(im2)
            term2 = np.nansum(term2)

            Cr = np.divide(np.nansum(corr_temp), np.sqrt(np.multiply(term1, term2)))
            N = sum(~np.isnan(corr_temp))
            Cr_spatial = len(np.where(~np.isnan(corr_temp)))*np.divide(corr_temp, np.sqrt(np.multiply(term1,term2)))
            Cr_m = len(np.where(~np.isnan(corr_temp)))*np.divide(np.nanmean(corr_temp), np.sqrt(np.multiply(term1, term2)))
            Cr_std = len(np.where(~np.isnan(corr_temp)))*np.divide(np.nanstd(corr_temp), np.sqrt(np.multiply(term1,term2)))

            Cr_kk.append(Cr)
            N_kk.append(N)
            Cr_spatial_kk.append(Cr_spatial)
            Cr_m_kk.append(Cr_m)
            Cr_std_kk.append(Cr_std)

        Cr_jj.append(Cr_kk)
        N_jj.append(N_kk)
        Cr_spatial_jj.update({jj:Cr_spatial_kk})
        Cr_m_jj.append(Cr_m_kk)
        Cr_std_jj.append(Cr_std_kk)

    return(r_bins, Cr_jj, N_jj, Cr_spatial_jj, Cr_m_jj, Cr_std_jj)


In [ ]:
def spatial_coord_NanNorm(piv_val, X_scaled, Y_scaled, r_bin_size):
    'Calculate all possible distances within a FOV'
    dists, r_bins = get_dists(X_scaled, Y_scaled, r_bin_size)

    Cr_jj = []
    N_jj = []

    for jj in range(len(r_bins)):
        if jj == 1:
            r, s, t, u = np.where(dists == r_bins[jj])
        else:
            r, s, t, u = np.where((dists > r_bins[jj-1]) & (dists < r_bins[jj]))

        Cr_kk = []
        N_kk = []
        for kk in range(len(piv_val)):
            im = piv_val[kk]
            im1 = []
            im2 = []
            for q in range(len(r)):
                im_1 = im[r[q], s[q]]
                im_2 = im[t[q], u[q]]
                im1.append(im_1)
                im2.append(im_2)

            # makes sure the terms have same number of unmasked values
            mask = np.divide(np.multiply(im1, im2), np.multiply(im1, im2))
            im1 = np.multiply(mask, im1)
            im2 = np.multiply(mask, im2)

            corr_temp = np.multiply(im1, im2)
            term1 = np.nansum(np.square(im1))
            term2 = np.nansum(np.square(im2))
            Cr = np.divide(np.nansum(corr_temp), np.sqrt(np.multiply(term1, term2)))
            N = sum(~np.isnan(corr_temp))

            Cr_kk.append(Cr)
            N_kk.append(N)

        Cr_jj.append(Cr_kk)
        N_jj.append(N_kk)
    
    # get mean Cr for plotting functions
    Cr_mean = np.nanmean(Cr_jj, axis=1)
    
    return(Cr_mean, r_bins, Cr_jj, N_jj)
    

In [ ]:
def spatial_coord_vector_NaNnorm(U_scaled, V_scaled, X_scaled, Y_scaled, r_bin_size):
    'Calculate all possible distances within a FOV'
    dists, r_bins = get_dists(X_scaled, Y_scaled, r_bin_size)

    Cr_jj = []
    N_jj = []
    Cr_spatial_jj={}
    Cr_m_jj = []
    Cr_std_jj = []

    for jj in range(len(r_bins)):
        if jj == 1:
            r, s, t, w = np.where(dists == r_bins[jj])
        else:
            r, s, t, w = np.where((dists > r_bins[jj-1]) & (dists < r_bins[jj]))

        Cr_kk = []
        N_kk = []
        for kk in range(len(U_scaled)):
            U_im = U_scaled[kk]
            V_im = V_scaled[kk]

            U_im1 = []
            U_im2 = []
            V_im1 = []
            V_im2 = []
            for q in range(len(r)):
                U_im_1 = im[r[q], s[q]]
                U_im_2 = im[t[q], u[q]]
                V_im_1 = im[r[q], s[q]]
                V_im_2 = im[t[q], u[q]]
                U_im1.append(U_im_1)
                U_im2.append(U_im_2)
                V_im1.append(V_im_1)
                V_im2.append(V_im_2)

            # makes sure the terms have same number of ~unmasked values
            mask = np.multiply(np.divide(U_im1, U_im1), np.divide(U_im2, U_im2), 
                               np.divide(V_im1, V_im1), np.divide(V_im2, V_im2))
            U_im1 = np.multiply(mask, U_im1)
            U_im2 = np.multiply(mask, U_im2)
            V_im1 = np.multiply(mask, V_im1)
            V_im2 = np.multiply(mask, V_im2)

            u_corr_temp = np.multiply(U_im1, U_im2)    
            v_corr_temp = np.multiply(V_im1, V_im2)    
            corr_temp = u_corr_temp + v_corr_temp

            u_term1 = np.nansum(np.square(U_im1))
            u_term2 = np.nansum(np.square(U_im2))
            v_term1 = np.nansum(np.square(V_im1))
            v_term2 = np.nansum(np.square(V_im2))
            term1 = u_term1 + v_term1
            term2 = u_term2 + v_term2

            Cr = np.divide(np.nansum(corr_temp), np.sqrt(np.multiply(term1, term2)))
            N = sum(sum(~np.isnan(corr_temp)))

            Cr_kk.append(Cr)
            N_kk.append(N)
        Cr_jj.append(Cr_kk)
        N_jj.append(N_kk)
        
    return(Cr_jj, N_jj, r_bins)
    

Piezo1-cKO PIV Analysis

In [ ]:
# Set the pixelsize
pixel_size= 0.6442*4
# Set the timestep between frames
delta_t = 5

# Set which basefolder to cycle through for analysis
folder="F:/Jesse/cKO/cKO_PIV/"
# folder = "F:/Jesse/tdTomato+Drug/Yoda1_PIV/"
settings.save_path = folder
images_list = sorted(glob(os.path.join(folder,"Nuclei", "*.tif")))
binary_list =  sorted(glob(os.path.join(folder,"Binary", "*.tif")))

# Doublecheck that our image list is matched to our binary list
if len(images_list) != len(binary_list):
    raise ValueError('Video length mismatch')

angles_dict={}
speed_dict = {}
ang_dev_all={}

rots=[]
angles_rot_dict={}

X_dict = {}
Y_dict = {}
U_dict = {}
V_dict = {}
v_rho_dict = {}
v_theta_dict = {}

# Create folder to save PIV results to
settings.save_path = os.path.join(settings.save_path, "Open_PIV_results")
if not os.path.exists(settings.save_path):
    os.makedirs(settings.save_path)

for i in tqdm(range(len(images_list)), position=1, leave=True):
    
    'General Housekeeping:'
    # Grab the filename from filepath for later use
    sfx = images_list[i].split('\\')
    file_name = sfx[-1][:-4] 
    
    nuclei_image = pims.TiffStack(images_list[i]) # Import image stack of nuclei_image[i]
    binary_image = pims.TiffStack(binary_list[i]) # Import corresponding image mask[i]
    
    # Create empty arrays for OpenPIV fields to be appended to
    U_raw=[]
    V_raw=[]
    X_raw=[]
    Y_raw=[]
    counter=0
    
    'Calculate multipass PIV Vector field for each frame pair in an image stack'
    for image in tqdm(range(len(nuclei_image)-1), position=0, leave=True):
        # Preprocessing of images
        img_a = piv_pre.normalize_array(nuclei_image[image])
        img_b = piv_pre.normalize_array(nuclei_image[image+1])
        img_a_pre = exposure.equalize_adapthist(img_a, 
                                               kernel_size = None, 
                                               clip_limit = 0.05, 
                                               nbins = 256)    
        img_b_pre = exposure.equalize_adapthist(img_b, 
                                               kernel_size = None, 
                                               clip_limit = 0.05, 
                                               nbins = 256)
        img_a_pre = piv_pre.instensity_cap(img_a_pre, 4)
        img_b_pre = piv_pre.instensity_cap(img_b_pre, 4)

        # Mask the nuclei image using binarized image. White should denote regions to keep, Black is masked
        settings.frame_pattern_a = cv.bitwise_and(cv.convertScaleAbs(img_a_pre),
                                                  ~binary_image[image])
        settings.frame_pattern_b = cv.bitwise_and(cv.convertScaleAbs(img_b_pre),
                                                  ~binary_image[image+1])
        mask_a = binary_image[image]
        mask_b = binary_image[image+1]
        x, y, u, v = piv(settings, file_name, mask_a, mask_b, counter)
        counter=counter+1 #counter is for the image file save information
        U_raw.append(u) #velocity vector in x direction
        V_raw.append(v) #velocity vector in y direction
        X_raw.append(x) #vector x possition
        Y_raw.append(y) #vector y position
    
    'Scale X,Y,U,V information from pixels to microns'
    X_scaled = np.array(X_raw)*pixel_size
    Y_scaled = np.array(Y_raw)*pixel_size
    U_scaled = np.array(U_raw)*(pixel_size/delta_t)
    V_scaled = np.array(V_raw)*(pixel_size/delta_t)
    
    'Calculate speed of vectors'
    speed_vec = np.square(U_scaled)+np.square(V_scaled)
    speed_dict.update({file_name:speed_vec})

    'Transform to Polar Coordinate'
    rho = np.sqrt(np.square(X_scaled)+np.square(Y_scaled))
    # v_rho or r_hat is the radial velocity
    v_rho = np.divide((np.multiply(U_scaled, X_scaled) + np.multiply(V_scaled, Y_scaled)), rho)
    # v_theta or theta_hat is the tangential velocity
    v_theta = np.divide((np.multiply(-U_scaled, Y_scaled) + np.multiply(V_scaled, X_scaled)), rho)
    # this is the angle of the piv vector
    angles = np.arctan2(v_theta, v_rho)
    
    'Calculate Angular Deviation'
    angle_dev_image=[]
    ang=[]
    for img in range(len(angles)):
        # We will be ignoring any values==0 or nan as these would just be noise
        ang.append(angles[img][~(angles[img]==0)])
        ang[img] = ang[img][~np.isnan(ang[img])]
        ang_dev = calc_ang_dev(ang[img])
        angle_dev_image.append(ang_dev)
    ang_dev_all.update({file_name:np.array(angle_dev_image)})
    angles_dict.update({file_name:angles})
    
    'Find the rotation angle to account for variance in scratch angle'
    rot_angle = straighten_scratch(binary_image)
    rots.append(rot_angle)
    X_rot, Y_rot = rotate_matrix(X_scaled, Y_scaled, rot_angle)
    U_rot, V_rot = rotate_matrix(U_scaled, V_scaled, rot_angle)
    
    'Polar Coordinates of Rotated Matrix'
    rho_rot = np.sqrt(np.square(X_rot)+np.square(Y_rot))
    v_rho_rot = np.divide((np.multiply(U_rot, X_rot) + np.multiply(V_rot, Y_rot)), rho_rot)
    v_theta_rot = np.divide((np.multiply(-U_rot, Y_rot) + np.multiply(V_rot, X_rot)), rho_rot)
    angles_rot = np.arctan2(v_theta_rot, v_rho_rot)
    angles_rot_dict.update({file_name:angles_rot})
    
    'Save X, Y, U, V, Rho_h, Theta_h info for later analysis'
    X_dict.update({file_name:X_scaled})
    Y_dict.update({file_name:Y_scaled})
    U_dict.update({file_name:U_scaled})
    V_dict.update({file_name:V_scaled})
    v_rho_dict.update({file_name:v_rho})
    v_theta_dict.update({file_name:v_theta})

Direct_Var_df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in ang_dev_all.items() ]))

# Save dictionaries as pickle files
date = datetime.now().strftime("%Y_%m_%d_")
pickle.dump(X_dict, builtins.open(f"{date}Con&cKO_PIV_X_dict.p", "wb"), pickle.HIGHEST_PROTOCOL)
pickle.dump(Y_dict, builtins.open(f"{date}Con&cKO_PIV_Y_dict.p", "wb"), pickle.HIGHEST_PROTOCOL)
pickle.dump(U_dict, builtins.open(f"{date}Con&cKO_PIV_U_dict.p", "wb"), pickle.HIGHEST_PROTOCOL)
pickle.dump(V_dict, builtins.open(f"{date}Con&cKO_PIV_V_dict.p", "wb"), pickle.HIGHEST_PROTOCOL)
pickle.dump(v_rho_dict, builtins.open(f"{date}Con&cKO_PIV_vRho_dict.p", "wb"), pickle.HIGHEST_PROTOCOL)
pickle.dump(v_theta_dict, builtins.open(f"{date}Con&cKO_PIV_vTheta_dict.p", "wb"), pickle.HIGHEST_PROTOCOL)
pickle.dump(ang_dev_all, builtins.open(f"{date}Con&cKO_PIV_AngDev_dict.p", "wb"), pickle.HIGHEST_PROTOCOL)
pickle.dump(angles_dict, builtins.open(f"{date}Con&cKO_PIV_Angles_dict.p", "wb"), pickle.HIGHEST_PROTOCOL)
pickle.dump(angles_rot_dict, builtins.open(f"{date}Con&cKO_Angles_Rot_dict.p", "wb"), pickle.HIGHEST_PROTOCOL)
pickle.dump(speed_dict, builtins.open(f"{date}Con&cKO_PIV_Speed_dict.p", "wb"), pickle.HIGHEST_PROTOCOL)

In [ ]:
rad_vel_Cr_mean_dict={}
rad_vel_r_bins_dict={}
rad_vel_Cr_dict={}
rad_vel_N_dict={}
key_hole = list(X_dict.keys())
for k in tqdm(range(len(key_hole)), position=0, leave=True):
    rad_vel_Cr_mean, rad_vel_r_bins, rad_vel_Cr, rad_vel_N = spatial_coord_NanNorm(v_rho_dict[key_hole[k]], 
                                                                                    X_dict[key_hole[k]], 
                                                                                    Y_dict[key_hole[k]], 15)
    rad_vel_Cr_mean_dict.update({key_hole[k]:rad_vel_Cr_mean})
    rad_vel_r_bins_dict.update({key_hole[k]:rad_vel_r_bins})
    rad_vel_Cr_dict.update({key_hole[k]:rad_vel_Cr})
    rad_vel_N_dict.update({key_hole[k]:rad_vel_N})

# Save dictionaries as pickle files
date = datetime.now().strftime("%Y_%m_%d_")
pickle.dump(rad_vel_Cr_mean_dict, builtins.open(f"{date}Con&cKO_PIV_RadVel_Cr_Mean_dict.p", "wb"), pickle.HIGHEST_PROTOCOL)
pickle.dump(rad_vel_r_bins_dict, builtins.open(f"{date}Con&cKO_PIV_RadVel_rbins_dict.p", "wb"), pickle.HIGHEST_PROTOCOL)
pickle.dump(rad_vel_Cr_dict, builtins.open(f"{date}Con&cKO_PIV_RadVel_Cr_dict.p", "wb"), pickle.HIGHEST_PROTOCOL)
pickle.dump(rad_vel_N_dict, builtins.open(f"{date}Con&cKO_PIV_RadVel_N_dict.p", "wb"), pickle.HIGHEST_PROTOCOL)

In [ ]:
# Load Pickle Files
X_dict = pickle.load(builtins.open("2022_05_12_Con&cKO_PIV_X_dict.p", "rb"))
Y_dict = pickle.load(builtins.open("2022_05_12_Con&cKO_PIV_Y_dict.p", "rb"))
U_dict = pickle.load(builtins.open("2022_05_12_Con&cKO_PIV_U_dict.p", "rb"))
V_dict = pickle.load(builtins.open("2022_05_12_Con&cKO_PIV_V_dict.p", "rb"))
v_rho_dict = pickle.load(builtins.open("2022_05_12_Con&cKO_PIV_vRho_dict.p", "rb"))
v_theta_dict = pickle.load(builtins.open("2022_05_12_Con&cKO_PIV_vTheta_dict.p", "rb"))
ang_dev_all = pickle.load(builtins.open("2022_05_12_Con&cKO_PIV_AngDev_dict.p", "rb"))
angles_dict = pickle.load(builtins.open("2022_05_12_Con&cKO_PIV_Angles_dict.p", "rb"))
angles_rot_dict = pickle.load(builtins.open("2022_05_12_Con&cKO_Angles_Rot_dict.p", "rb"))
speed_dict = pickle.load(builtins.open("2022_05_12_Con&cKO_PIV_Speed_dict.p", "rb"))
rad_vel_Cr_mean_dict = pickle.load(builtins.open("2022_05_15_Con&cKO_PIV_RadVel_Cr_Mean_dict.p", "rb"))
rad_vel_r_bins_dict = pickle.load(builtins.open("2022_05_15_Con&cKO_PIV_RadVel_rbins_dict.p", "rb"))
rad_vel_Cr_dict = pickle.load(builtins.open("2022_05_15_Con&cKO_PIV_RadVel_Cr_dict.p", "rb"))
rad_vel_N_dict = pickle.load(builtins.open("2022_05_15_Con&cKO_PIV_RadVel_N_dict.p", "rb"))

In [ ]:
# Mean of each mean Cr for a FOV

Con_cKO_Cr_mean=[]
cKO_Cr_mean=[]

key_hole = list(rad_vel_Cr_mean_dict.keys())
for k in range(len(key_hole)):
    if key_hole[k].split('_')[-2]=='cKO' or key_hole[k].split('_')[-4]=='cKO' or key_hole[k].split('_')[-3]=='cKO':
        cKO_Cr_mean.append(rad_vel_Cr_mean_dict[key_hole[k]])
    elif key_hole[k].split('_')[-2]=='Con' or key_hole[k].split('_')[-4]=='Con' or key_hole[k].split('_')[-3]=='Con':
        Con_cKO_Cr_mean.append(rad_vel_Cr_mean_dict[key_hole[k]])
    else:
        print(key_hole[k])
        raise ValueError('Genotype mismatch')

Con_Cr_avg = pd.DataFrame(Con_cKO_Cr_mean).T.mean(axis=1)
Con_Cr_sem = pd.DataFrame(Con_cKO_Cr_mean).T.sem(axis=1)
cKO_Cr_avg = pd.DataFrame(cKO_Cr_mean).T.mean(axis=1)
cKO_Cr_sem = pd.DataFrame(cKO_Cr_mean).T.sem(axis=1)

In [ ]:
max_len = 114
max_key = ""
for key in rad_vel_r_bins_dict:
    cur_len = len(rad_vel_r_bins_dict[key])
    if cur_len==max_len:
        max_key = key
        max_len = cur_len
print(max_key)

In [ ]:
# Calculate Local Coordination for Con/cKO
col_names = rad_vel_r_bins_dict['317_2021_03_27_batch66_Pos8_cKO_top_prediction']
df_1 = pd.DataFrame(cKO_Cr_mean)
df_1.columns = col_names
cKO_Cr_150 = df_1[150.0]

col_names = rad_vel_r_bins_dict['317_2021_03_27_batch66_Pos40_Con_bot_prediction']
df_2 = pd.DataFrame(Con_cKO_Cr_mean)
df_2.columns = col_names
Con_cKO_Cr_150 = df_2[150.0]

In [ ]:
cKO_RadVel_All = pd.concat([pd.DataFrame({'R Bins':rad_vel_r_bins_dict['312_03_25_batch66_Pos10_Con_bot']}), 
           pd.DataFrame({'Con Radial Cr Mean':Con_Cr_avg}), 
           pd.DataFrame({'Con Radial Cr SEM':Con_Cr_sem}),
           pd.DataFrame({'cKO Radial Cr Mean':cKO_Cr_avg}),
           pd.DataFrame({'cKO Radial Cr SEM':cKO_Cr_sem})], axis=1)
date = datetime.now().strftime("%Y_%m_%d")
cKO_RadVel_All.to_excel(f'{date}_cKO_mean_Radial_Cr.xlsx')

In [ ]:
cKO_Cr_fig = go.Figure()

cKO_Cr_fig.add_trace(go.Scatter(x= rad_vel_r_bins_dict['312_03_25_batch66_Pos10_Con_bot'][0:35], y= pd.DataFrame(Con_cKO_Cr_mean).T.mean(axis=1), name='Con(cKO)', line_color='#000000', error_y=dict(type='data', array = pd.DataFrame(Con_cKO_Cr_mean).T.sem(axis=1), visible=True)))
cKO_Cr_fig.add_trace(go.Scatter(x= rad_vel_r_bins_dict['312_03_25_batch66_Pos10_Con_bot'][0:35], y= pd.DataFrame(cKO_Cr_mean).T.mean(axis=1), name='cKO', line_color='#6a0dad', error_y =dict(type='data', array = pd.DataFrame(cKO_Cr_mean).T.sem(axis=1) , visible=True)))
cKO_Cr_fig.update_layout(title="Spatial Coordination (Autocorrelation of Radial Velocity)", xaxis_title="Δr(µm)", yaxis_title="C(Δr)", legend_title="Condition")
cKO_Cr_fig.show()

In [ ]:
cKO_Cr_mean_anova = []
for col in range(6, np.shape(pd.DataFrame(Con_cKO_Cr_mean))[1]):
    stat, p = stats.f_oneway(pd.DataFrame(Con_cKO_Cr_mean)[col], pd.DataFrame(cKO_Cr_mean)[col])
    ks_stat, ks_p = stats.ks_2samp(pd.DataFrame(Con_cKO_Cr_mean)[col], pd.DataFrame(cKO_Cr_mean)[col])
#     mw_stat, mw_p = stats.mannwhitneyu(pd.DataFrame(Con_cKO_Cr_mean)[col], pd.DataFrame(cKO_Cr_mean)[col])
    d = cohend(pd.DataFrame(Con_cKO_Cr_mean)[col], pd.DataFrame(cKO_Cr_mean)[col])
    cKO_Cr_mean_anova.append([stat, p, ks_stat, ks_p, mw_stat, mw_p, d])

date = datetime.now().strftime("%Y_%m_%d_")
cKO_Con_mean_stats = pd.DataFrame(cKO_Cr_mean_anova, columns= ['ANOVA stat', 'ANOVA p','KS Stat',' KS p','MW Stat',' MW p', 'Cohens d'])
cKO_Con_mean_stats.to_excel(f'{date}_cKOvCon_Cr_mean_Stats.xlsx')

In [ ]:
angles_Cr_mean_dict={}
angles_r_bins_dict={}
angles_Cr_dict={}
angles_N_dict={}
key_hole = list(X_dict.keys())
for k in tqdm(range(len(key_hole)), position=0, leave=True):
    angles_Cr_mean, angles_r_bins, angles_Cr, angles_N = spatial_coord_NanNorm(v_theta_dict[key_hole[k]], 
                                        X_dict[key_hole[k]], 
                                        Y_dict[key_hole[k]], 15)
    angles_Cr_mean_dict.update({key_hole[k]:tan_vel_Cr_mean})
    angles_r_bins_dict.update({key_hole[k]:tan_vel_r_bins})
    angles_Cr_dict.update({key_hole[k]:tan_vel_Cr})
    angles_N_dict.update({key_hole[k]:tan_vel_N})

# Save dictionaries as pickle files
date = datetime.now().strftime("%Y_%m_%d_")
pickle.dump(angles_Cr_mean_dict, builtins.open(f"{date}Con&cKO_PIV_Angles_Cr_Mean_dict.p", "wb"), pickle.HIGHEST_PROTOCOL)
pickle.dump(angles_r_bins_dict, builtins.open(f"{date}Con&cKO_PIV_Angles_rbins_dict.p", "wb"), pickle.HIGHEST_PROTOCOL)
pickle.dump(angles_Cr_dict, builtins.open(f"{date}Con&cKO_PIV_Angles_Cr_dict.p", "wb"), pickle.HIGHEST_PROTOCOL)
pickle.dump(angles_N_dict, builtins.open(f"{date}Con&cKO_PIV_Angles_N_dict.p", "wb"), pickle.HIGHEST_PROTOCOL)

In [ ]:
angles_Cr_mean_dict = pickle.load(builtins.open("2022_05_19_Con&cKO_PIV_Angles_Cr_Mean_dict.p", "rb"))
angles_r_bins_dict = pickle.load(builtins.open("2022_05_19_Con&cKO_PIV_Angles_rbins_dict.p", "rb"))
angles_Cr_dict = pickle.load(builtins.open("2022_05_19_Con&cKO_PIV_Angles_Cr_dict.p", "rb"))
angles_N_dict = pickle.load(builtins.open("2022_05_19_Con&cKO_PIV_Angles_N_dict.p", "rb"))

In [ ]:
# Mean of each mean Cr for a FOV

Con_cKO_angles_Cr_mean = []
cKO_angles_Cr_mean = []

key_hole = list(angles_Cr_mean_dict.keys())
for k in range(len(key_hole)):
    if key_hole[k].split('_')[-2]=='cKO' or key_hole[k].split('_')[-4]=='cKO' or key_hole[k].split('_')[-3]=='cKO':
        cKO_angles_Cr_mean.append(angles_Cr_mean_dict[key_hole[k]])
    elif key_hole[k].split('_')[-2]=='Con' or key_hole[k].split('_')[-4]=='Con' or key_hole[k].split('_')[-3]=='Con':
        Con_cKO_angles_Cr_mean.append(angles_Cr_mean_dict[key_hole[k]])
    else:
        print(key_hole[k])
        raise ValueError('Genotype mismatch')

Angles_Con_Cr_avg = pd.DataFrame(Con_cKO_angles_Cr_mean).T.mean(axis=1)
Angles_Con_Cr_sem = pd.DataFrame(Con_cKO_angles_Cr_mean).T.sem(axis=1)
Angles_cKO_Cr_avg = pd.DataFrame(cKO_angles_Cr_mean).T.mean(axis=1)
Angles_cKO_Cr_sem = pd.DataFrame(cKO_angles_Cr_mean).T.sem(axis=1)

In [ ]:
Con_cKO=[]
cKO=[]
speeds=[]
key_hole = list(speed_dict.keys())
for k in range(len(key_hole)):
    speeds.append(np.array(speed_dict[key_hole[k]])[~np.isnan(speed_dict[key_hole[k]])])
    speeds[k] = speeds[k][~(speeds[k]==0)]
    if key_hole[k].split('_')[-2]=='cKO' or key_hole[k].split('_')[-4]=='cKO' or key_hole[k].split('_')[-3]=='cKO':
        cKO.append(speeds[k])
    elif key_hole[k].split('_')[-2]=='Con' or key_hole[k].split('_')[-4]=='Con' or key_hole[k].split('_')[-3]=='Con':
        Con_cKO.append(speeds[k])
    else:
        print(key_hole[k])
        raise ValueError('Genotype mismatch')
cKO_speeds = np.concatenate(cKO)
Con_cKO_speeds = np.concatenate(Con_cKO)

In [ ]:
Con_cKO=[]
Con_names=[]
cKO=[]
cKO_names=[]

ang=[]
key_hole = list(angles_rot_dict.keys())
for k in range(len(key_hole)):
    ang.append(np.array(angles_rot_dict[key_hole[k]])[~np.isnan(angles_rot_dict[key_hole[k]])])
    ang[k] = ang[k][~(ang[k]==0)]
    ang[k] = ang[k]+((3*np.pi)/4)
    for i in range(len(ang[k])):
        if ang[k][i] >  np.pi:
            ang[k][i] = ang[k][i]-(2*np.pi)
        elif ang[k][i] <  -np.pi:
            ang[k][i] = ang[k][i]+(2*np.pi)
    if key_hole[k].split('_')[-2]=='cKO' or key_hole[k].split('_')[-4]=='cKO' or key_hole[k].split('_')[-3]=='cKO':
        cKO.append(ang[k])
        cKO_names.append(key_hole[k])
    elif key_hole[k].split('_')[-2]=='Con' or key_hole[k].split('_')[-4]=='Con' or key_hole[k].split('_')[-3]=='Con':
        Con_cKO.append(ang[k])
        Con_names.append(key_hole[k])
    else:
        print(key_hole[k])
        raise ValueError('Genotype mismatch')
cKO_angles = np.concatenate(cKO)
Con_cKO_angles = np.concatenate(Con_cKO)

In [ ]:
date = datetime.now().strftime("%Y_%m_%d_%I_%M_%p")
Con_cKO_angles_df = pd.DataFrame(Con_cKO_angles)
cKO_angles_df = pd.DataFrame(cKO_angles)
Con_cKO_angles_df.to_csv(f'{date}_ConcKO_angles.csv')
cKO_angles_df.to_csv(f'{date}_cKO_angles.csv')

In [ ]:
cKO_angles = pd.read_csv ("C:/Users/17605/Downloads/2022_04_30_02_06_PM_cKO_angles.csv")
Con_cKO_angles = pd.read_csv ("C:/Users/17605/Downloads/2022_04_30_02_06_PM_ConcKO_angles.csv")
Con_cKO_angles.drop(['Unnamed: 0'], axis=1,inplace=True)
cKO_angles.drop(['Unnamed: 0'], axis=1,inplace=True)
cKO_angles.rename({"0":"Vector Direction (Degrees)"}, axis=1, inplace=True)
Con_cKO_angles.rename({"0":"Vector Direction (Degrees)"}, axis=1, inplace=True)

In [ ]:
Con_cKO = pd.DataFrame(np.degrees(np.array(Con_cKO_angles)), columns=['Direction'])
cKO = pd.DataFrame(np.degrees(np.array(cKO_angles)), columns=['Direction'])
Con_cKO["Condition"]="Con(cKO)"
cKO["Condition"]="cKO"
cKO_v_Con_df = pd.concat([Con_cKO, cKO], axis=0)
cKO_v_Con_df.head()

In [ ]:
fig = px.histogram(cKO_v_Con_df, x="Direction",
             color="Condition",
             labels={"Direction":"Vector Direction (Degrees)"},
             color_discrete_map = {'Control(cKO)':'#BAB0AC', 'cKO':'#A777F1'},
             barmode="overlay", 
             nbins=30,
             range_x=[-180,180],
             range_y=[0,15],
             histnorm= "percent",
             marginal="box")
date = datetime.now().strftime("%Y_%m_%d_%I_%M_%p")

fig.write_html(f'{date}_ConcCKO_Overlay_directionality_Figure.html', auto_open=True)


In [ ]:
df = pd.DataFrame(np.degrees(np.array(Con_cKO_angles)), columns=['Vector Direction (Degree)'])

fig = px.histogram(df, x='Vector Direction (Degree)',  
                   title= 'Con Directionality', 
                   histnorm= "percent", 
                   nbins=30, 
                   range_y=[0,10],
                   range_x=[-180,180],
                   color_discrete_sequence = ['grey'])
fig.update_layout(font=dict(size=18))

date = datetime.now().strftime("%Y_%m_%d_%I_%M_%p")

fig.write_html(f'{date}_Con_directionality_Figure.html', auto_open=True)


In [ ]:
df = pd.DataFrame(np.degrees(np.array(cKO_angles)), columns=['Vector Direction (Degree)'])

fig = px.histogram(df, x='Vector Direction (Degree)',  
                   title= 'cKO Directionality', 
                   histnorm= "percent", 
                   nbins=30, 
                   range_y=[0,10],
                   range_x=[-180,180],
                   color_discrete_sequence = ['purple'])
fig.update_layout(font=dict(size=18))
date = datetime.now().strftime("%Y_%m_%d_%I_%M_%p")
fig.write_html(f'{date}_cKO_directionality_Figure.html', auto_open=True)

In [ ]:
Con_cKO_dev=[]
cKO_dev=[]
ConcKO_names=[]
cKO_names=[]

key_hole = list(ang_dev_all.keys())
for k in range(len(key_hole)):
    if key_hole[k].split('_')[-3]=='Con' or key_hole[k].split('_')[-4]=='Con'  or key_hole[k].split('_')[-2]=='Con':
        Con_cKO_dev.append(np.nanmean(ang_dev_all[key_hole[k]]))
        ConcKO_names.append(key_hole[k])

    elif key_hole[k].split('_')[-3]=='cKO' or key_hole[k].split('_')[-4]=='cKO' or  key_hole[k].split('_')[-2]=='cKO':
        cKO_dev.append(np.nanmean(ang_dev_all[key_hole[k]]))
        cKO_names.append(key_hole[k])

    else:
        print(key_hole[k])
        raise ValueError('Genotype mismatch')

In [ ]:
con_v_cKO_angdev = pd.concat([pd.DataFrame(ConcKO_names, columns=['Filename']),
                        pd.DataFrame(Con_cKO_dev, columns=['Control(cKO)']),
                        pd.DataFrame(cKO_names, columns=['Filename']),
                        pd.DataFrame(cKO_dev, columns=['cKO'])], axis=1)
con_v_cKO_angdev.to_excel(f'{date}cKO_v_Con_AngDev.xlsx')

In [ ]:
cko_ang_dev = pd.concat([pd.DataFrame(Con_cKO_dev, columns=['Control(cKO)']),
                      pd.DataFrame(cKO_dev, columns=['cKO'])], axis=1)
cKO_ang_dev_dabest = dabest.load(cko_ang_dev, idx=("Control(cKO)", "cKO"), resamples=5000)
cKO_ang_dev_dabest.cohens_d

In [ ]:
color_pal={'Control(cKO)':'black', 'cKO': 'purple'}
cKO_ang_dev_dabest.cohens_d.plot(swarm_label='Angular Deviation',float_contrast=False, custom_palette=color_pal, raw_marker_size=3);

Yoda1 PIV Analysis

In [ ]:
# Set the pixelsize
pixel_size= 0.6442*4
# Set the timestep between frames
delta_t = 5

# Set which basefolder to cycle through for analysis

folder = "F:/Jesse/tdTomato+Drug/Yoda1_PIV/"
settings.save_path = folder
images_list = sorted(glob(os.path.join(folder,"Nuclei", "*.tif")))
binary_list =  sorted(glob(os.path.join(folder,"Binary", "*.tif")))

# Doublecheck that our image list is matched to our binary list
if len(images_list) != len(binary_list):
    raise ValueError('Video length mismatch')

angles_dict={}
speed_dict = {}
ang_dev_all={}

rots=[]
angles_rot_dict={}

X_dict = {}
Y_dict = {}
U_dict = {}
V_dict = {}
v_rho_dict = {}
v_theta_dict = {}

# Create folder to save PIV results to
settings.save_path = os.path.join(settings.save_path, "Open_PIV_results")
if not os.path.exists(settings.save_path):
    os.makedirs(settings.save_path)

for i in tqdm(range(len(images_list)), position=1, leave=True):
    
    'General Housekeeping:'
    # Grab the filename from filepath for later use
    sfx = images_list[i].split('\\')
    file_name = sfx[-1][:-4] 
    
    nuclei_image = pims.TiffStack(images_list[i]) # Import image stack of nuclei_image[i]
    binary_image = pims.TiffStack(binary_list[i]) # Import corresponding image mask[i]
    
    # Create empty arrays for OpenPIV fields to be appended to
    U_raw=[]
    V_raw=[]
    X_raw=[]
    Y_raw=[]
    counter=0
    
    'Calculate multipass PIV Vector field for each frame pair in an image stack'
    for image in tqdm(range(len(nuclei_image)-1), position=0, leave=True):
        # Preprocessing of images
        img_a = piv_pre.normalize_array(nuclei_image[image])
        img_b = piv_pre.normalize_array(nuclei_image[image+1])
        img_a_pre = exposure.equalize_adapthist(img_a, 
                                               kernel_size = None, 
                                               clip_limit = 0.05, 
                                               nbins = 256)    
        img_b_pre = exposure.equalize_adapthist(img_b, 
                                               kernel_size = None, 
                                               clip_limit = 0.05, 
                                               nbins = 256)
        img_a_pre = piv_pre.instensity_cap(img_a_pre, 4)
        img_b_pre = piv_pre.instensity_cap(img_b_pre, 4)

        # Mask the nuclei image using binarized image. White should denote regions to keep, Black is masked
        settings.frame_pattern_a = cv.bitwise_and(cv.convertScaleAbs(img_a_pre),
                                                  ~binary_image[image])
        settings.frame_pattern_b = cv.bitwise_and(cv.convertScaleAbs(img_b_pre),
                                                  ~binary_image[image+1])
        mask_a = binary_image[image]
        mask_b = binary_image[image+1]
        x, y, u, v = piv(settings, file_name, mask_a, mask_b, counter)
        counter=counter+1 #counter is for the image file save information
        U_raw.append(u) #velocity vector in x direction
        V_raw.append(v) #velocity vector in y direction
        X_raw.append(x) #vector x possition
        Y_raw.append(y) #vector y position
    
    'Scale X,Y,U,V information from pixels to microns'
    X_scaled = np.array(X_raw)*pixel_size
    Y_scaled = np.array(Y_raw)*pixel_size
    U_scaled = np.array(U_raw)*(pixel_size/delta_t)
    V_scaled = np.array(V_raw)*(pixel_size/delta_t)
    
    'Calculate speed of vectors'
    speed_vec = np.square(U_scaled)+np.square(V_scaled)
    speed_dict.update({file_name:speed_vec})

    'Transform to Polar Coordinate'
    rho = np.sqrt(np.square(X_scaled)+np.square(Y_scaled))
    # v_rho or r_hat is the radial velocity
    v_rho = np.divide((np.multiply(U_scaled, X_scaled) + np.multiply(V_scaled, Y_scaled)), rho)
    # v_theta or theta_hat is the tangential velocity
    v_theta = np.divide((np.multiply(-U_scaled, Y_scaled) + np.multiply(V_scaled, X_scaled)), rho)
    # this is the angle of the piv vector
    angles = np.arctan2(v_theta, v_rho)
    
    'Calculate Angular Deviation'
    angle_dev_image=[]
    ang=[]
    for img in range(len(angles)):
        # We will be ignoring any values==0 or nan as these would just be noise
        ang.append(angles[img][~(angles[img]==0)])
        ang[img] = ang[img][~np.isnan(ang[img])]
        ang_dev = calc_ang_dev(ang[img])
        angle_dev_image.append(ang_dev)
    ang_dev_all.update({file_name:np.array(angle_dev_image)})
    angles_dict.update({file_name:angles})
    
    'Find the rotation angle to account for variance in scratch angle'
    rot_angle = straighten_scratch(binary_image)
    rots.append(rot_angle)
    X_rot, Y_rot = rotate_matrix(X_scaled, Y_scaled, rot_angle)
    U_rot, V_rot = rotate_matrix(U_scaled, V_scaled, rot_angle)
    
    'Polar Coordinates of Rotated Matrix'
    rho_rot = np.sqrt(np.square(X_rot)+np.square(Y_rot))
    v_rho_rot = np.divide((np.multiply(U_rot, X_rot) + np.multiply(V_rot, Y_rot)), rho_rot)
    v_theta_rot = np.divide((np.multiply(-U_rot, Y_rot) + np.multiply(V_rot, X_rot)), rho_rot)
    angles_rot = np.arctan2(v_theta_rot, v_rho_rot)
    angles_rot_dict.update({file_name:angles_rot})
    
    'Save X, Y, U, V, Rho_h, Theta_h info for later analysis'
    X_dict.update({file_name:X_scaled})
    Y_dict.update({file_name:Y_scaled})
    U_dict.update({file_name:U_scaled})
    V_dict.update({file_name:V_scaled})
    v_rho_dict.update({file_name:v_rho})
    v_theta_dict.update({file_name:v_theta})

Direct_Var_df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in ang_dev_all.items() ]))

date = datetime.now().strftime("%Y_%m_%d_")
pickle.dump(X_dict, builtins.open(f"{date}DMSO&Yoda1_PIV_X_dict.p", "wb"), pickle.HIGHEST_PROTOCOL)
pickle.dump(Y_dict, builtins.open(f"{date}DMSO&Yoda1_PIV_Y_dict.p", "wb"), pickle.HIGHEST_PROTOCOL)
pickle.dump(U_dict, builtins.open(f"{date}DMSO&Yoda1_PIV_U_dict.p", "wb"), pickle.HIGHEST_PROTOCOL)
pickle.dump(V_dict, builtins.open(f"{date}DMSO&Yoda1_PIV_V_dict.p", "wb"), pickle.HIGHEST_PROTOCOL)
pickle.dump(v_rho_dict, builtins.open(f"{date}DMSO&Yoda1_PIV_vRho_dict.p", "wb"), pickle.HIGHEST_PROTOCOL)
pickle.dump(v_theta_dict, builtins.open(f"{date}DMSO&Yoda1_PIV_vTheta_dict.p", "wb"), pickle.HIGHEST_PROTOCOL)
pickle.dump(ang_dev_all, builtins.open(f"{date}DMSO&Yoda1_PIV_AngDev_dict.p", "wb"), pickle.HIGHEST_PROTOCOL)
pickle.dump(angles_dict, builtins.open(f"{date}DMSO&Yoda1_PIV_Angles_dict.p", "wb"), pickle.HIGHEST_PROTOCOL)
pickle.dump(angles_rot_dict, builtins.open(f"{date}DMSO&Yoda1_Angles_Rot_dict.p", "wb"), pickle.HIGHEST_PROTOCOL)
pickle.dump(speed_dict, builtins.open(f"{date}DMSO&Yoda1_PIV_Speed_dict.p", "wb"), pickle.HIGHEST_PROTOCOL)

In [ ]:
# Load Pickle Files
X_dict = pickle.load(builtins.open("2022_06_21_DMSO&Yoda1_PIV_X_dict.p", "rb"))
Y_dict = pickle.load(builtins.open("2022_06_21_DMSO&Yoda1_PIV_Y_dict.p", "rb"))
U_dict = pickle.load(builtins.open("2022_06_21_DMSO&Yoda1_PIV_U_dict.p", "rb"))
V_dict = pickle.load(builtins.open("2022_06_21_DMSO&Yoda1_PIV_V_dict.p", "rb"))
v_rho_dict = pickle.load(builtins.open("2022_06_21_DMSO&Yoda1_PIV_vRho_dict.p", "rb"))
v_theta_dict = pickle.load(builtins.open("2022_06_21_DMSO&Yoda1_PIV_vTheta_dict.p", "rb"))
ang_dev_all = pickle.load(builtins.open("2022_06_21_DMSO&Yoda1_PIV_AngDev_dict.p", "rb"))
angles_dict = pickle.load(builtins.open("2022_06_21_DMSO&Yoda1_PIV_Angles_dict.p", "rb"))
angles_rot_dict = pickle.load(builtins.open("2022_06_21_DMSO&Yoda1_Angles_Rot_dict.p", "rb"))
speed_dict = pickle.load(builtins.open("2022_06_21_DMSO&Yoda1_PIV_Speed_dict.p", "rb"))
rad_vel_Cr_mean_dict = pickle.load(builtins.open("2022_06_22_DMSO&Yoda1_PIV_RadVel_Cr_Mean_dict.p", "rb"))
rad_vel_r_bins_dict = pickle.load(builtins.open("2022_06_22_DMSO&Yoda1_PIV_RadVel_rbins_dict.p", "rb"))
rad_vel_Cr_dict = pickle.load(builtins.open("2022_06_22_DMSO&Yoda1_PIV_RadVel_Cr_dict.p", "rb"))
rad_vel_N_dict = pickle.load(builtins.open("2022_06_22_DMSO&Yoda1_PIV_RadVel_N_dict.p", "rb"))


In [ ]:
# Calculate Spatial Autocorrelation of Radial Velocity
rad_vel_Cr_mean_dict={}
rad_vel_r_bins_dict={}
rad_vel_Cr_dict={}
rad_vel_N_dict={}
key_hole = list(X_dict.keys())
for k in tqdm(range(len(key_hole)), position=0, leave=True):
    rad_vel_Cr_mean, rad_vel_r_bins, rad_vel_Cr, rad_vel_N = spatial_coord_NanNorm(v_rho_dict[key_hole[k]], 
                                                                                    X_dict[key_hole[k]], 
                                                                                    Y_dict[key_hole[k]], 15)
    rad_vel_Cr_mean_dict.update({key_hole[k]:rad_vel_Cr_mean})
    rad_vel_r_bins_dict.update({key_hole[k]:rad_vel_r_bins})
    rad_vel_Cr_dict.update({key_hole[k]:rad_vel_Cr})
    rad_vel_N_dict.update({key_hole[k]:rad_vel_N})

# Save dictionaries as pickle files
date = datetime.now().strftime("%Y_%m_%d_")
pickle.dump(rad_vel_Cr_mean_dict, builtins.open(f"{date}DMSO&Yoda1_PIV_RadVel_Cr_Mean_dict.p", "wb"), pickle.HIGHEST_PROTOCOL)
pickle.dump(rad_vel_r_bins_dict, builtins.open(f"{date}DMSO&Yoda1_PIV_RadVel_rbins_dict.p", "wb"), pickle.HIGHEST_PROTOCOL)
pickle.dump(rad_vel_Cr_dict, builtins.open(f"{date}DMSO&Yoda1_PIV_RadVel_Cr_dict.p", "wb"), pickle.HIGHEST_PROTOCOL)
pickle.dump(rad_vel_N_dict, builtins.open(f"{date}DMSO&Yoda1_PIV_RadVel_N_dict.p", "wb"), pickle.HIGHEST_PROTOCOL)

In [ ]:
# Calculate Mean Spatial Coordination and SEM for Plotting

DMSO_Cr_mean = []
Yoda1_Cr_mean = []

key_hole = list(rad_vel_Cr_dict.keys())
for k in range(len(key_hole)):
    if key_hole[k].split('_')[-2]=='DMSO' or key_hole[k].split('_')[-3]=='DMSO':
        DMSO_Cr_mean.append(rad_vel_Cr_mean_dict[key_hole[k]])
    elif key_hole[k].split('_')[-2]=='Yoda1' or key_hole[k].split('_')[-2]=='Y1' or key_hole[k].split('_')[-3]=='Yoda1' or key_hole[k].split('_')[-4]=='Yoda1':
        Yoda1_Cr_mean.append(rad_vel_Cr_mean_dict[key_hole[k]])
    else:
        print(key_hole[k])
        raise ValueError('Genotype mismatch')

DMSO_Cr_avg = pd.DataFrame(DMSO_Cr_mean).T.mean(axis=1)
DMSO_Cr_sem = pd.DataFrame(DMSO_Cr_mean).T.sem(axis=1)
Yoda1_Cr_avg = pd.DataFrame(Yoda1_Cr_mean).T.mean(axis=1)
Yoda1_Cr_sem = pd.DataFrame(Yoda1_Cr_mean).T.sem(axis=1)

In [ ]:
max_len = 112
max_key = ""
for key in rad_vel_r_bins_dict:
    cur_len = len(rad_vel_r_bins_dict[key])
    if cur_len==max_len:
        max_key = key
        max_len = cur_len
print(max_key)

In [ ]:
# Calculate Local Coordination for DMSO/Yoda1
col_names = rad_vel_r_bins_dict['263_2020_08_20_Scratch_Pos7_Yoda1_top']
df_1 = pd.DataFrame(Yoda1_Cr_mean)
df_1.columns = col_names
Yoda1_Cr_150 = df_1[150.0]

col_names = rad_vel_r_bins_dict['332_2021_05_02_batch70_Pos20_skin3_Y1_top']
df_2 = pd.DataFrame(DMSO_Cr_mean)
df_2.columns = col_names
DMSO_Cr_150 = df_2[150.0]

In [ ]:
# Plotting of Mean spatial autocorrelation of Radial Velocity component
Yoda1_Cr_fig = go.Figure()

Yoda1_Cr_fig.add_trace(go.Scatter(x= rad_vel_r_bins_dict['332_2021_05_02_batch70_DMSO_Pos3_top'][0:35], y= pd.DataFrame(DMSO_Cr_mean).T.mean(axis=1), name='DMSO', line_color='#000000', error_y=dict(type='data', array = pd.DataFrame(DMSO_Cr_mean).T.sem(axis=1), visible=True)))
Yoda1_Cr_fig.add_trace(go.Scatter(x= rad_vel_r_bins_dict['332_2021_05_02_batch70_DMSO_Pos3_top'][0:35], y= pd.DataFrame(Yoda1_Cr_mean).T.mean(axis=1), name='Yoda1', line_color='#ff0000', error_y =dict(type='data', array = pd.DataFrame(Yoda1_Cr_mean).T.sem(axis=1) , visible=True)))
Yoda1_Cr_fig.update_layout(title="Spatial Coordination (Autocorrelation of Radial Velocity)", xaxis_title="Δr(µm)", yaxis_title="C(Δr)", legend_title="Condition")
Yoda1_Cr_fig.show()

In [ ]:
DMSO_RadVel_All = pd.concat([pd.DataFrame({'R Bins':rad_vel_r_bins_dict['332_2021_05_02_batch70_DMSO_Pos3_top']}), 
           pd.DataFrame({'DMSO Radial Cr Mean':DMSO_Cr_avg}), 
           pd.DataFrame({'DMSO Radial Cr SEM':DMSO_Cr_sem}),
           pd.DataFrame({'Yoda1 Radial Cr Mean':Yoda1_Cr_avg}),
           pd.DataFrame({'Yoda1 Radial Cr SEM':Yoda1_Cr_sem})], axis=1)
date = datetime.now().strftime("%Y_%m_%d")
DMSO_RadVel_All.to_excel(f'{date}_DMSO&Yoda1_mean_Radial_Cr.xlsx')

In [ ]:
Yoda1_Cr_mean_anova = []
for col in range(0,np.shape(pd.DataFrame(DMSO_Cr_mean))[1]):
    stat, p = stats.f_oneway(pd.DataFrame(DMSO_Cr_mean)[col], pd.DataFrame(Yoda1_Cr_mean)[col])
    ks_stat, ks_p = stats.ks_2samp(pd.DataFrame(DMSO_Cr_mean)[col], pd.DataFrame(Yoda1_Cr_mean)[col])
    if len(pd.DataFrame(DMSO_Cr_mean)[col].dropna())>0 & len(pd.DataFrame(Yoda1_Cr_mean)[col].dropna())>0:
        mw_stat, mw_p = stats.mannwhitneyu(pd.DataFrame(DMSO_Cr_mean)[col].dropna(), pd.DataFrame(Yoda1_Cr_mean)[col].dropna())
    else:
        mw_stat, mw_p = 0, 0
    d = cohend(pd.DataFrame(DMSO_Cr_mean)[col], pd.DataFrame(Yoda1_Cr_mean)[col])
    Yoda1_Cr_mean_anova.append([stat, p, ks_stat, ks_p, mw_stat, mw_p, d])

date = datetime.now().strftime("%Y_%m_%d_")
cKO_Con_mean_stats = pd.DataFrame(Yoda1_Cr_mean_anova, columns= ['ANOVA stat', 'ANOVA p','KS Stat',' KS p','MW Stat',' MW p', 'Cohens d'])
cKO_Con_mean_stats.to_excel(f'{date}DMSO&Yoda1_Cr_mean_Stats.xlsx')

In [ ]:
DMSO=[]
DMSO_names=[]
Yoda1=[]
Yoda1_names=[]

ang=[]
key_hole = list(angles_rot_dict.keys())
for k in range(len(key_hole)):
    ang.append(np.array(angles_rot_dict[key_hole[k]])[~np.isnan(angles_rot_dict[key_hole[k]])])
    ang[k] = ang[k][~(ang[k]==0)]
    ang[k] = ang[k]+((3*np.pi)/4)
    for i in range(len(ang[k])):
        if ang[k][i] >  np.pi:
            ang[k][i] = ang[k][i]-(2*np.pi)
        elif ang[k][i] <  -np.pi:
            ang[k][i] = ang[k][i]+(2*np.pi)
    if key_hole[k].split('_')[-2]=='DMSO' or key_hole[k].split('_')[-3]=='DMSO':
        DMSO.append(ang[k])
        DMSO_names.append(key_hole[k])
    elif key_hole[k].split('_')[-2]=='Yoda1' or key_hole[k].split('_')[-2]=='Y1' or key_hole[k].split('_')[-3]=='Yoda1' or key_hole[k].split('_')[-4]=='Yoda1':
        Yoda1.append(ang[k])
        Yoda1_names.append(key_hole[k])
    else:
        print(key_hole[k])
        raise ValueError('Genotype mismatch')
DMSO_angles = np.concatenate(DMSO)
Yoda1_angles = np.concatenate(Yoda1)

In [ ]:
date = datetime.now().strftime("%Y_%m_%d_%I_%M_%p")
DMSO_angles_df = pd.DataFrame(DMSO_angles)
Yoda1_angles_df = pd.DataFrame(Yoda1_angles)
DMSO_angles_df.to_csv(f'{date}_DMSO_angles.csv')
Yoda1_angles_df.to_csv(f'{date}_Yoda1_angles.csv')

In [ ]:
DMSO = pd.DataFrame(np.degrees(np.array(DMSO_angles)), columns=['Direction'])
Yoda1 = pd.DataFrame(np.degrees(np.array(Yoda1_angles)), columns=['Direction'])
DMSO["Condition"]="DMSO"
Yoda1["Condition"]="Yoda1"
DMSO_v_Yoda1_df = pd.concat([DMSO, Yoda1], axis=0)
DMSO_v_Yoda1_df.head()

Make Figures for Yoda1

In [ ]:
fig = px.histogram(DMSO_v_Yoda1_df, x="Direction",
             color="Condition",
             labels={"Direction":"Vector Direction (Degrees)"},
             color_discrete_map = {'DMSO':'#BAB0AC', 'Yoda1':'#ff0000'},
             barmode="overlay", 
             nbins=30,
             range_x=[-180,180],
             range_y=[0,10],
             histnorm= "percent",
             marginal="box")
date = datetime.now().strftime("%Y_%m_%d_%I_%M_%p")

fig.write_html(f'{date}_DMSO&Yoda1_Overlay_directionality_Figure.html', auto_open=True)


In [ ]:
df = pd.DataFrame(np.degrees(np.array(DMSO_angles)), columns=['Vector Direction (Degree)'])

fig = px.histogram(df, x='Vector Direction (Degree)',  
                   title= 'DMSO Directionality', 
                   histnorm= "percent", 
                   nbins=30, 
                   range_y=[0,10],
                   range_x=[-180,180],
                   color_discrete_sequence = ['grey'])
fig.update_layout(font=dict(size=18))

date = datetime.now().strftime("%Y_%m_%d_%I_%M_%p")

fig.write_html(f'{date}_DMSO_directionality_Figure.html', auto_open=True)


In [ ]:
df = pd.DataFrame(np.degrees(np.array(Yoda1_angles)), columns=['Vector Direction (Degree)'])

fig = px.histogram(df, x='Vector Direction (Degree)',  
                   title= 'Yoda1 Directionality', 
                   histnorm= "percent", 
                   nbins=30, 
                   range_y=[0,10],
                   range_x=[-180,180],
                   color_discrete_sequence = ['red'])
fig.update_layout(font=dict(size=18))
date = datetime.now().strftime("%Y_%m_%d_%I_%M_%p")
fig.write_html(f'{date}_Yoda1_directionality_Figure.html', auto_open=True)

In [ ]:
# Calculate Angular Deviation
Yoda1_dev=[]
DMSO_dev=[]
DMSO_names=[]
Yoda1_names=[]

key_hole = list(ang_dev_all.keys())
for k in range(len(key_hole)):
    if key_hole[k].split('_')[-2]=='Yoda1' or key_hole[k].split('_')[-2]=='Y1' or key_hole[k].split('_')[-3]=='Yoda1' or key_hole[k].split('_')[-4]=='Yoda1':
        Yoda1_dev.append(np.nanmean(ang_dev_all[key_hole[k]]))
        Yoda1_names.append(key_hole[k])
    elif key_hole[k].split('_')[-2]=='DMSO' or key_hole[k].split('_')[-3]=='DMSO':
        DMSO_dev.append(np.nanmean(ang_dev_all[key_hole[k]]))
        DMSO_names.append(key_hole[k])
    else:
        print(key_hole[k])
        raise ValueError('Genotype mismatch')

In [ ]:
DMSO_v_Yoda1_angdev = pd.concat([pd.DataFrame(DMSO_names, columns=['Filename']),
                        pd.DataFrame(DMSO_dev, columns=['DMSO']),
                        pd.DataFrame(Yoda1_names, columns=['Filename']),
                        pd.DataFrame(Yoda1_dev, columns=['Yoda1'])], axis=1)
DMSO_v_Yoda1_angdev.to_excel(f'{date}DMSO_v_Yoda1_AngDev.xlsx')

In [ ]:
y1_ang_dev = pd.concat([pd.DataFrame(DMSO_dev, columns=['DMSO']),
                      pd.DataFrame(Yoda1_dev, columns=['Yoda1'])], axis=1)
y1_ang_dev_dabest = dabest.load(y1_ang_dev, idx=("DMSO", "Yoda1"), resamples=5000)
y1_ang_dev_dabest.cohens_d

In [ ]:
y1_ang_dev_dabest.cohens_d.results

In [ ]:
color_pal={'DMSO':'black', 'Yoda1': 'red'}
y1_ang_dev_dabest.cohens_d.plot(swarm_label='Angular Deviation',float_contrast=False, custom_palette=color_pal, raw_marker_size=3);

Piezo1-Gof PIV Analysis

In [ ]:
# Set the pixelsize
pixel_size= 0.6442*4
# Set the timestep between frames
delta_t = 5

# Set which basefolder to cycle through for analysis

folder = "F:/Jesse/GoF/GoF_PIV/"
settings.save_path = folder
images_list = sorted(glob(os.path.join(folder,"Nuclei", "*.tif")))
binary_list =  sorted(glob(os.path.join(folder,"Binary", "*.tif")))

# Doublecheck that our image list is matched to our binary list
if len(images_list) != len(binary_list):
    raise ValueError('Video length mismatch')

angles_dict={}
speed_dict = {}
ang_dev_all={}

rots=[]
angles_rot_dict={}

X_dict = {}
Y_dict = {}
U_dict = {}
V_dict = {}
v_rho_dict = {}
v_theta_dict = {}

# Create folder to save PIV results to
settings.save_path = os.path.join(settings.save_path, "Open_PIV_results")
if not os.path.exists(settings.save_path):
    os.makedirs(settings.save_path)

for i in tqdm(range(len(images_list)), position=1, leave=True):
    
    'General Housekeeping:'
    # Grab the filename from filepath for later use
    sfx = images_list[i].split('\\')
    file_name = sfx[-1][:-4] 
    
    nuclei_image = pims.TiffStack(images_list[i]) # Import image stack of nuclei_image[i]
    binary_image = pims.TiffStack(binary_list[i]) # Import corresponding image mask[i]
    
    # Create empty arrays for OpenPIV fields to be appended to
    U_raw=[]
    V_raw=[]
    X_raw=[]
    Y_raw=[]
    counter=0
    
    'Calculate multipass PIV Vector field for each frame pair in an image stack'
    for image in tqdm(range(len(nuclei_image)-1), position=0, leave=True):
        # Preprocessing of images
        img_a = piv_pre.normalize_array(nuclei_image[image])
        img_b = piv_pre.normalize_array(nuclei_image[image+1])
        img_a_pre = exposure.equalize_adapthist(img_a, 
                                               kernel_size = None, 
                                               clip_limit = 0.05, 
                                               nbins = 256)    
        img_b_pre = exposure.equalize_adapthist(img_b, 
                                               kernel_size = None, 
                                               clip_limit = 0.05, 
                                               nbins = 256)
        img_a_pre = piv_pre.instensity_cap(img_a_pre, 4)
        img_b_pre = piv_pre.instensity_cap(img_b_pre, 4)

        # Mask the nuclei image using binarized image. White should denote regions to keep, Black is masked
        settings.frame_pattern_a = cv.bitwise_and(cv.convertScaleAbs(img_a_pre),
                                                  ~binary_image[image])
        settings.frame_pattern_b = cv.bitwise_and(cv.convertScaleAbs(img_b_pre),
                                                  ~binary_image[image+1])
        mask_a = binary_image[image]
        mask_b = binary_image[image+1]
        x, y, u, v = piv(settings, file_name, mask_a, mask_b, counter)
        counter=counter+1 #counter is for the image file save information
        U_raw.append(u) #velocity vector in x direction
        V_raw.append(v) #velocity vector in y direction
        X_raw.append(x) #vector x possition
        Y_raw.append(y) #vector y position
    
    'Scale X,Y,U,V information from pixels to microns'
    X_scaled = np.array(X_raw)*pixel_size
    Y_scaled = np.array(Y_raw)*pixel_size
    U_scaled = np.array(U_raw)*(pixel_size/delta_t)
    V_scaled = np.array(V_raw)*(pixel_size/delta_t)
    
    'Calculate speed of vectors'
    speed_vec = np.square(U_scaled)+np.square(V_scaled)
    speed_dict.update({file_name:speed_vec})

    'Transform to Polar Coordinate'
    rho = np.sqrt(np.square(X_scaled)+np.square(Y_scaled))
    # v_rho or r_hat is the radial velocity
    v_rho = np.divide((np.multiply(U_scaled, X_scaled) + np.multiply(V_scaled, Y_scaled)), rho)
    # v_theta or theta_hat is the tangential velocity
    v_theta = np.divide((np.multiply(-U_scaled, Y_scaled) + np.multiply(V_scaled, X_scaled)), rho)
    # this is the angle of the piv vector
    angles = np.arctan2(v_theta, v_rho)
    
    'Calculate Angular Deviation'
    angle_dev_image=[]
    ang=[]
    for img in range(len(angles)):
        # We will be ignoring any values==0 or nan as these would just be noise
        ang.append(angles[img][~(angles[img]==0)])
        ang[img] = ang[img][~np.isnan(ang[img])]
        ang_dev = calc_ang_dev(ang[img])
        angle_dev_image.append(ang_dev)
    ang_dev_all.update({file_name:np.array(angle_dev_image)})
    angles_dict.update({file_name:angles})
    
    'Find the rotation angle to account for variance in scratch angle'
    rot_angle = straighten_scratch(binary_image)
    rots.append(rot_angle)
    X_rot, Y_rot = rotate_matrix(X_scaled, Y_scaled, rot_angle)
    U_rot, V_rot = rotate_matrix(U_scaled, V_scaled, rot_angle)
    
    'Polar Coordinates of Rotated Matrix'
    rho_rot = np.sqrt(np.square(X_rot)+np.square(Y_rot))
    v_rho_rot = np.divide((np.multiply(U_rot, X_rot) + np.multiply(V_rot, Y_rot)), rho_rot)
    v_theta_rot = np.divide((np.multiply(-U_rot, Y_rot) + np.multiply(V_rot, X_rot)), rho_rot)
    angles_rot = np.arctan2(v_theta_rot, v_rho_rot)
    angles_rot_dict.update({file_name:angles_rot})
    
    'Save X, Y, U, V, Rho_h, Theta_h info for later analysis'
    X_dict.update({file_name:X_scaled})
    Y_dict.update({file_name:Y_scaled})
    U_dict.update({file_name:U_scaled})
    V_dict.update({file_name:V_scaled})
    v_rho_dict.update({file_name:v_rho})
    v_theta_dict.update({file_name:v_theta})

Direct_Var_df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in ang_dev_all.items() ]))

date = datetime.now().strftime("%Y_%m_%d_")
pickle.dump(X_dict, builtins.open(f"{date}Con&GoF_PIV_X_dict.p", "wb"), pickle.HIGHEST_PROTOCOL)
pickle.dump(Y_dict, builtins.open(f"{date}Con&GoF_PIV_Y_dict.p", "wb"), pickle.HIGHEST_PROTOCOL)
pickle.dump(U_dict, builtins.open(f"{date}Con&GoF_PIV_U_dict.p", "wb"), pickle.HIGHEST_PROTOCOL)
pickle.dump(V_dict, builtins.open(f"{date}Con&GoF_PIV_V_dict.p", "wb"), pickle.HIGHEST_PROTOCOL)
pickle.dump(v_rho_dict, builtins.open(f"{date}Con&GoF_PIV_vRho_dict.p", "wb"), pickle.HIGHEST_PROTOCOL)
pickle.dump(v_theta_dict, builtins.open(f"{date}Con&GoF_PIV_vTheta_dict.p", "wb"), pickle.HIGHEST_PROTOCOL)
pickle.dump(ang_dev_all, builtins.open(f"{date}Con&GoF_PIV_AngDev_dict.p", "wb"), pickle.HIGHEST_PROTOCOL)
pickle.dump(angles_dict, builtins.open(f"{date}Con&GoF_PIV_Angles_dict.p", "wb"), pickle.HIGHEST_PROTOCOL)
pickle.dump(angles_rot_dict, builtins.open(f"{date}Con&GoF_Angles_Rot_dict.p", "wb"), pickle.HIGHEST_PROTOCOL)
pickle.dump(speed_dict, builtins.open(f"{date}Con&GoF_PIV_Speed_dict.p", "wb"), pickle.HIGHEST_PROTOCOL)

In [ ]:
# 6 Pass
X_dict = pickle.load(builtins.open("2022_06_22_Con&GoF_PIV_X_dict.p", "rb"))
Y_dict = pickle.load(builtins.open("2022_06_22_Con&GoF_PIV_Y_dict.p", "rb"))
U_dict = pickle.load(builtins.open("2022_06_22_Con&GoF_PIV_U_dict.p", "rb"))
V_dict = pickle.load(builtins.open("2022_06_22_Con&GoF_PIV_V_dict.p", "rb"))
v_rho_dict = pickle.load(builtins.open("2022_06_22_Con&GoF_PIV_vRho_dict.p", "rb"))
v_theta_dict = pickle.load(builtins.open("2022_06_22_Con&GoF_PIV_vTheta_dict.p", "rb"))
ang_dev_all = pickle.load(builtins.open("2022_06_22_Con&GoF_PIV_AngDev_dict.p", "rb"))
angles_dict = pickle.load(builtins.open("2022_06_22_Con&GoF_PIV_Angles_dict.p", "rb"))
angles_rot_dict = pickle.load(builtins.open("2022_06_22_Con&GoF_Angles_Rot_dict.p", "rb"))
speed_dict = pickle.load(builtins.open("2022_06_22_Con&GoF_PIV_Speed_dict.p", "rb"))
rad_vel_Cr_mean_dict = pickle.load(builtins.open("2022_06_23_04_01_PM_Con&GoF_PIV_RadVel_Cr_Mean_dict.p", "rb"))
rad_vel_r_bins_dict = pickle.load(builtins.open("2022_06_23_04_01_PM_Con&GoF_PIV_RadVel_rbins_dict.p", "rb"))
rad_vel_Cr_dict = pickle.load(builtins.open("2022_06_23_04_01_PM_Con&GoF_PIV_RadVel_Cr_dict.p", "rb"))
rad_vel_N_dict = pickle.load(builtins.open("2022_06_23_04_01_PM_Con&GoF_PIV_RadVel_N_dict.p", "rb"))

In [ ]:
# Calculate Spatial Autocorrelation of Radial Velocity Componenent
rad_vel_Cr_mean_dict={}
rad_vel_r_bins_dict={}
rad_vel_Cr_dict={}
rad_vel_N_dict={}
key_hole = list(X_dict.keys())
for k in tqdm(range(len(key_hole)), position=0, leave=True):
    rad_vel_Cr_mean, rad_vel_r_bins, rad_vel_Cr, rad_vel_N = spatial_coord_NanNorm(v_rho_dict[key_hole[k]], 
                                                                                    X_dict[key_hole[k]], 
                                                                                    Y_dict[key_hole[k]], 15)
    rad_vel_Cr_mean_dict.update({key_hole[k]:rad_vel_Cr_mean})
    rad_vel_r_bins_dict.update({key_hole[k]:rad_vel_r_bins})
    rad_vel_Cr_dict.update({key_hole[k]:rad_vel_Cr})
    rad_vel_N_dict.update({key_hole[k]:rad_vel_N})

# Save dictionaries as pickle files
date = datetime.now().strftime("%Y_%m_%d_%I_%M_%p_")
pickle.dump(rad_vel_Cr_mean_dict, builtins.open(f"{date}Con&GoF_PIV_RadVel_Cr_Mean_dict.p", "wb"), pickle.HIGHEST_PROTOCOL)
pickle.dump(rad_vel_r_bins_dict, builtins.open(f"{date}Con&GoF_PIV_RadVel_rbins_dict.p", "wb"), pickle.HIGHEST_PROTOCOL)
pickle.dump(rad_vel_Cr_dict, builtins.open(f"{date}Con&GoF_PIV_RadVel_Cr_dict.p", "wb"), pickle.HIGHEST_PROTOCOL)
pickle.dump(rad_vel_N_dict, builtins.open(f"{date}Con&GoF_PIV_RadVel_N_dict.p", "wb"), pickle.HIGHEST_PROTOCOL)

In [ ]:
# Calculate Mean Spatial Coordination for each genotype

Con_Cr_mean = []
GoF_Cr_mean = []

key_hole = list(rad_vel_Cr_dict.keys())
for k in range(len(key_hole)):
    if key_hole[k].split('_')[-2]=='Con':
        Con_Cr_mean.append(rad_vel_Cr_mean_dict[key_hole[k]])
    elif key_hole[k].split('_')[-2]=='GoF':
        GoF_Cr_mean.append(rad_vel_Cr_mean_dict[key_hole[k]])
    else:
        print(key_hole[k])
        raise ValueError('Genotype mismatch')

Con_Cr_avg = pd.DataFrame(Con_Cr_mean).T.mean(axis=1)
Con_Cr_sem = pd.DataFrame(Con_Cr_mean).T.sem(axis=1)
GoF_Cr_avg = pd.DataFrame(GoF_Cr_mean).T.mean(axis=1)
GoF_Cr_sem = pd.DataFrame(GoF_Cr_mean).T.sem(axis=1)

In [ ]:
max_len = 120
max_key = ""
for key in rad_vel_r_bins_dict:
    cur_len = len(rad_vel_r_bins_dict[key])
    if cur_len==max_len:
        max_key = key
        max_len = cur_len
print(max_key)

In [ ]:
# Calculate Local Coordination
col_names = rad_vel_r_bins_dict['285_2021_01_26_Pos7_GoF_top']
df = pd.DataFrame(GoF_Cr_mean)
df.columns = col_names
GoF_Cr_150 = df[150.0]

col_names = rad_vel_r_bins_dict['329_2021_04_10_Pos25_Con_top']
df = pd.DataFrame(Con_Cr_mean)
df.columns = col_names
ConGoF_Cr_150 = df[150.0]

In [ ]:
# Plot GoF Spatial Coordination Figure
GoF_Cr_fig = go.Figure()

GoF_Cr_fig.add_trace(go.Scatter(x= rad_vel_r_bins_dict['282_2021_01_25_Pos15_Con_bot'][0:35], y= pd.DataFrame(Con_Cr_mean).T.mean(axis=1), name='Con', line_color='#000000', error_y=dict(type='data', array = pd.DataFrame(Con_Cr_mean).T.sem(axis=1), visible=True)))
GoF_Cr_fig.add_trace(go.Scatter(x= rad_vel_r_bins_dict['282_2021_01_25_Pos15_Con_bot'][0:35], y= pd.DataFrame(GoF_Cr_mean).T.mean(axis=1), name='GoF', line_color='#006400', error_y =dict(type='data', array = pd.DataFrame(GoF_Cr_mean).T.sem(axis=1) , visible=True)))
GoF_Cr_fig.update_layout(title="Spatial Coordination (Autocorrelation of Radial Velocity)", xaxis_title="Δr(µm)", yaxis_title="C(Δr)", legend_title="Condition")
GoF_Cr_fig.show()

In [ ]:
GoF_RadVel_All = pd.concat([pd.DataFrame({'R Bins':rad_vel_r_bins_dict['282_2021_01_25_Pos15_Con_bot']}), 
           pd.DataFrame({'Con Radial Cr Mean':Con_Cr_avg}), 
           pd.DataFrame({'Con Radial Cr SEM':Con_Cr_sem}),
           pd.DataFrame({'GoF Radial Cr Mean':GoF_Cr_avg}),
           pd.DataFrame({'GoF Radial Cr SEM':GoF_Cr_sem})], axis=1)
date = datetime.now().strftime("%Y_%m_%d")
GoF_RadVel_All.to_excel(f'{date}_Con&GoF_mean_Radial_Cr.xlsx')

In [ ]:
GoF_Cr_mean_anova = []
for col in range(0,np.shape(pd.DataFrame(GoF_Cr_mean))[1]):
    stat, p = stats.f_oneway(pd.DataFrame(Con_Cr_mean)[col], pd.DataFrame(GoF_Cr_mean)[col])
    ks_stat, ks_p = stats.ks_2samp(pd.DataFrame(Con_Cr_mean)[col], pd.DataFrame(GoF_Cr_mean)[col])
    if len(pd.DataFrame(Con_Cr_mean)[col].dropna())>0 & len(pd.DataFrame(GoF_Cr_mean)[col].dropna())>0:
        mw_stat, mw_p = stats.mannwhitneyu(pd.DataFrame(Con_Cr_mean)[col].dropna(), pd.DataFrame(GoF_Cr_mean)[col].dropna())
    else:
        mw_stat, mw_p = 0, 0
    d = cohend(pd.DataFrame(Con_Cr_mean)[col], pd.DataFrame(GoF_Cr_mean)[col])
    GoF_Cr_mean_anova.append([stat, p, ks_stat, ks_p, mw_stat, mw_p, d])

date = datetime.now().strftime("%Y_%m_%d_")
GoF_mean_stats = pd.DataFrame(GoF_Cr_mean_anova, columns= ['ANOVA stat', 'ANOVA p','KS Stat',' KS p','MW Stat',' MW p', 'Cohens d'])
GoF_mean_stats.to_excel(f'{date}_Con&GoF_Cr_mean_Stats.xlsx')

In [ ]:
Con=[]
Con_names=[]
GoF=[]
GoF_names=[]

ang=[]
key_hole = list(angles_rot_dict.keys())
for k in range(len(key_hole)):
    ang.append(np.array(angles_rot_dict[key_hole[k]])[~np.isnan(angles_rot_dict[key_hole[k]])])
    ang[k] = ang[k][~(ang[k]==0)]
    ang[k] = ang[k]+((3*np.pi)/4)
    for i in range(len(ang[k])):
        if ang[k][i] >  np.pi:
            ang[k][i] = ang[k][i]-(2*np.pi)
        elif ang[k][i] <  -np.pi:
            ang[k][i] = ang[k][i]+(2*np.pi)
    if key_hole[k].split('_')[-2]=='Con':
        Con.append(ang[k])
        Con_names.append(key_hole[k])
    elif key_hole[k].split('_')[-2]=='GoF':
        GoF.append(ang[k])
        GoF_names.append(key_hole[k])
    else:
        print(key_hole[k])
        raise ValueError('Genotype mismatch')
Con_angles = np.concatenate(Con)
GoF_angles = np.concatenate(GoF)

In [ ]:
date = datetime.now().strftime("%Y_%m_%d_%I_%M_%p")
Con_angles_df = pd.DataFrame(Con_angles)
GoF_angles_df = pd.DataFrame(GoF_angles)
Con_angles_df.to_csv(f'{date}_Con(GoF)_angles.csv')
GoF_angles_df.to_csv(f'{date}_GoF_angles.csv')

In [ ]:
Con = pd.DataFrame(np.degrees(np.array(Con_angles)), columns=['Direction'])
GoF = pd.DataFrame(np.degrees(np.array(GoF_angles)), columns=['Direction'])
Con["Condition"]="Con"
GoF["Condition"]="GoF"
Con_v_GoF_df = pd.concat([Con, GoF], axis=0)
Con_v_GoF_df.head()

In [ ]:
fig = px.histogram(Con_v_GoF_df, x="Direction",
             color = "Condition",
             labels = {"Direction":"Vector Direction (Degrees)"},
             color_discrete_map = {'Con':'#BAB0AC', 'GoF':'#006400'},
             barmode = "overlay", 
             nbins = 30,
             range_x = [-180,180],
             range_y = [0,10],
             histnorm = "percent",
             marginal = "box")
date = datetime.now().strftime("%Y_%m_%d_%I_%M_%p")

fig.write_html(f'{date}_Con&GoF_Overlay_directionality_Figure.html', auto_open=True)


In [ ]:
df = pd.DataFrame(np.degrees(np.array(Con_angles)), columns=['Vector Direction (Degree)'])

fig = px.histogram(df, x='Vector Direction (Degree)',  
                   title= 'Con Directionality', 
                   histnorm= "percent", 
                   nbins=30, 
                   range_y=[0,10],
                   range_x=[-180,180],
                   color_discrete_sequence = ['grey'])
fig.update_layout(font=dict(size=18))

date = datetime.now().strftime("%Y_%m_%d_%I_%M_%p")

fig.write_html(f'{date}_Con(GoF)_directionality_Figure.html', auto_open=True)


In [ ]:
df = pd.DataFrame(np.degrees(np.array(Yoda1_angles)), columns=['Vector Direction (Degree)'])

fig = px.histogram(df, x='Vector Direction (Degree)',  
                   title= 'GoF Directionality', 
                   histnorm= "percent", 
                   nbins=30, 
                   range_y=[0,10],
                   range_x=[-180,180],
                   color_discrete_sequence = ['green'])
fig.update_layout(font=dict(size=18))
date = datetime.now().strftime("%Y_%m_%d_%I_%M_%p")
fig.write_html(f'{date}_GoF_directionality_Figure.html', auto_open=True)

In [ ]:
ConGoF_dev=[]
ConGoF_names=[]
GoF_names=[]
GoF_dev=[]
ang=[]

Con_GoF_dev=[]
GoF_dev=[]
key_hole = list(ang_dev_all.keys())
for k in range(len(key_hole)):
    if key_hole[k].split('_')[-2]=='GoF':
        GoF_dev.append(np.nanmean(ang_dev_all[key_hole[k]]))
        GoF_names.append(key_hole[k])
    elif key_hole[k].split('_')[-2]=='Con':
        ConGoF_dev.append(np.nanmean(ang_dev_all[key_hole[k]]))
        ConGoF_names.append(key_hole[k])
    else:
        print(key_hole[k])
        raise ValueError('Genotype mismatch')

In [ ]:
Con_v_GoF_angdev = pd.concat([pd.DataFrame(ConGoF_names, columns=['Filename']),
                        pd.DataFrame(ConGoF_dev, columns=['Con']),
                        pd.DataFrame(GoF_names, columns=['Filename']),
                        pd.DataFrame(GoF_dev, columns=['GoF'])], axis=1)
Con_v_GoF_angdev.to_excel(f'{date}Con_v_GoF_AngDev.xlsx')

In [ ]:
gof_ang_dev = pd.concat([pd.DataFrame(ConGoF_dev, columns=['Con']),
                      pd.DataFrame(GoF_dev, columns=['GoF'])], axis=1)
gof_ang_dev_dabest = dabest.load(gof_ang_dev, idx=("Con", "GoF"), resamples=5000)
gof_ang_dev_dabest.cohens_d

In [ ]:
gof_ang_dev_dabest.cohens_d.results

In [ ]:
color_pal={'Con':'black', 'GoF': 'green'}
gof_ang_dev_dabest.cohens_d.plot(swarm_label='Angular Deviation',float_contrast=False, custom_palette=color_pal, raw_marker_size=3);

In [ ]:
Con_stack = pd.melt(Con_DV, var_name='Filename Con', value_name='Con DV')
GoF_stack = pd.melt(GoF_DV, var_name='Filename GoF', value_name='GoF DV')
Direc_Var_Pair = pd.concat([DMSO_stack, Yoda1_stack], axis=1)

Direc_Var_dabest = dabest.load(Direc_Var_Pair, idx=("Con DV", "GoF DV"), resamples=5000)
Direc_Var_dabest.cohens_d

In [ ]:
color_pal={'Con DV':'black', 'GoF DV': 'red'}
Direc_Var_dabest.cohens_d.plot(swarm_label='Direction Variance',float_contrast=False, custom_palette=color_pal, raw_marker_size=3);

Make Figures with All Data Combined

In [ ]:
# file_name = '262_2020_08_19_scratch_Pos2_DMSO_bot'
file_name = '262_2020_08_19_scratch_Pos7_Yoda1_top'
# file_name = '312_03_25_batch66_Pos26_cKO_bot'
# file_name = '312_03_25_batch66_Pos8_Con_top'
# file_name = '282_2021_01_25_Pos20_GoF_top'
# file_name = '327_2021_04_08_Pos0_GoF_top'
# file_name = '327_2021_04_08_Pos14_Con_top'

X_mode = stats.mode(X_dict[file_name],axis=0)[0][0]
Y_mode = stats.mode(Y_dict[file_name],axis=0)[0][0]
U_avg = np.mean(v_theta_dict[file_name], axis=0)
V_avg = np.mean(v_rho_dict[file_name], axis=0)

colors = np.arctan2(U_avg, V_avg)+((3*np.pi)/4)
for i in range(len(colors)):
    for k in range(len(colors[i])):
        if colors[i][k] >  np.pi:
            colors[i][k] = colors[i][k]-(2*np.pi)
        elif colors[i][k] <  -np.pi:
            colors[i][k] = colors[i][k]+(2*np.pi)

norm = Normalize()
norm.autoscale(colors)

data = V_avg
data_mean, data_std = np.nanmean(data), np.nanstd(data)
cut_off = data_std * 3
lower, upper = data_mean - cut_off, data_mean + cut_off
count = 0
for x in range(len(data)):
    for y in range(len(data[x])):
        if data[x][y] < lower or data[x][y] > upper:
            data[x][y] = np.nan
            count=count+1
            

vectorplot = plt.figure(figsize=(10,7))
plt.quiver(X_mode, Y_mode, U_avg, V_avg, degrees(colors), angles='xy', cmap= 'inferno', clim=[-180,180])
plt.colorbar(label="Direction (Degrees)", orientation="horizontal")
plt.show()

vectorplot.savefig(f'G:/Shared drives/Keratinocytes/06_Figures/02_Modeling/Figure PIV/{file_name}_6.svg', format='svg')

In [ ]:
all_ang_dev = pd.concat([pd.DataFrame(Con_cKO_dev, columns=['Control(cKO)']),
                         pd.DataFrame(cKO_dev, columns=['cKO']),
                         pd.DataFrame(ConGoF_dev, columns=['Control(GoF)']),
                         pd.DataFrame(GoF_dev, columns=['GoF']),
                         pd.DataFrame(DMSO_dev, columns=['DMSO']),
                         pd.DataFrame(Yoda1_dev, columns=['Yoda1'])], axis=1)
all_ang_dev_norm = all_ang_dev/np.sqrt(2) # norm so that 1 is most random
all_ang_dev_dabest = dabest.load(all_ang_dev_norm, idx = (("Control(cKO)", "cKO"),
                                                      ("Control(GoF)", "GoF"),
                                                         ("DMSO", "Yoda1")), resamples=5000)
all_ang_dev_dabest.cohens_d

color_pal = {'Control(cKO)':'grey', 'cKO': '#d1a4ff', 'DMSO':'black', 'Yoda1': 'red', 'Control(GoF)':'grey', 'GoF': '#bbffbb',}
edge_plot = all_ang_dev_dabest.cohens_d.plot(swarm_label='Angular Deviation',float_contrast=False, custom_palette=color_pal, raw_marker_size=7);

date = datetime.now().strftime("%Y_%m_%d_")
edge_plot.savefig(f"{date}_angle_deviation_all.svg", format='svg')

In [ ]:
all_ang_dev_dabest.cohens_d.results

In [ ]:
all_spat_coor = pd.concat([pd.DataFrame(Con_cKO_Cr_150).rename(columns={150.0:'Control(cKO)'}),
                           pd.DataFrame(cKO_Cr_150).rename(columns={150.0:'cKO'}),
                           pd.DataFrame(ConGoF_Cr_150).rename(columns={150.0:'Control(GoF)'}),
                           pd.DataFrame(GoF_Cr_150).rename(columns={150.0:'GoF'}),
                           pd.DataFrame(DMSO_Cr_150).rename(columns={150.0:'DMSO'}),
                           pd.DataFrame(Yoda1_Cr_150).rename(columns={150.0:'Yoda1'})], axis=1)
all_spat_coor_dabest = dabest.load(all_spat_coor, idx = (("Control(cKO)", "cKO"),
                                                      ("Control(GoF)", "GoF"),
                                                         ("DMSO", "Yoda1")), resamples=5000)
all_spat_coor_dabest.cohens_d

color_pal = {'Control(cKO)':'grey', 'cKO': '#d1a4ff', 'DMSO':'black', 'Yoda1': 'red', 'Control(GoF)':'grey', 'GoF': '#bbffbb',}
spat_coor_plot = all_spat_coor_dabest.cohens_d.plot(swarm_label='Local Coordination',float_contrast=False, custom_palette=color_pal, raw_marker_size=7);

date = datetime.now().strftime("%Y_%m_%d_")
spat_coor_plot.savefig(f"{date}_LocalCoordination_all.svg", format='svg')

In [ ]:
all_spat_coor_dabest.cohens_d.results